In [142]:
# Importa bibliotecas necessárias para manipulação de dados e criação de gráficos
import numpy as np
import pandas as pd
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
import unicodedata
import re
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import fpgrowth, association_rules
from IPython.display import display




In [143]:
#fazendo a leitura do arquivo da base de dados
df = pd.read_csv('vendas_modificado(in).csv');
df.head()

,id_da_compra,data,hora,cliente,produto,valor,quantidade,total,status,cidade,estado,pais,cep,frete,pagamento,vendedor,marca
0,13679,3/20/2021,23:35:51,LUCAS Araujo Kuhn,Queijo Mussarela,"R$ 16,87",13,239.31,Pagamento Confirmado,Niterói,RJ,Brasil,24000-000,20.0,Cartão de Crédito,SAMUEL HENRIQUE CAÇADOR,Porto Alegre
1,28070,10/30/2020,9:00:53,MICAEL SOUZA RONCETE,Molho de Tomate,"R$ 3,25",3,9.75,Pagamento Confirmado,Mariana,MG,Brasil,35420-000,0.0,Pix,MICAEL MALAQUIAS DE SOUZA OLIVEIRA,Fugini
2,47484,6/9/2021,15:30:28,FELIPE AUGUSTO NERY SILVA,Água Mineral,"R$ 1,63",9,36.67,Pagamento Confirmado,Cabo Frio,RJ,Brasil,28900-000,22.0,Transferência Bancária,HENRICO MATOS LIMA DA CUNHA,Minalba
3,20809,6/4/2022,8:41:23,LEVI RIBEIRO AMORIM,Carvão,"R$ 8,74",4,54.96,Em Separação,Campos dos Goytacazes,RJ,Brasil,28000-000,20.0,Pix,GABRIEL QUEIROZ DE AGUIAR,Marca-Brás
4,47123,5/4/2019,13:38:45,GABRIEL MATOS LIMA DA CUNHA,Café,"R$ 9,48",2,18.96,Em Separação,Conselheiro Lafaiete,MG,Brasil,36400-000,0.0,Cartão de Crédito,HENRICO MATOS LIMA DA CUNHA,3 Corações


In [144]:
#ordenar pelo id da compra de forma crescente
df_emOrdem = df.sort_values(by="id_da_compra")
display(df_emOrdem.head(10))

,id_da_compra,data,hora,cliente,produto,valor,quantidade,total,status,cidade,estado,pais,cep,frete,pagamento,vendedor,marca
119876,0,7/7/2019,11:05:44,PEDRO SILVA RAMOS,Sabonete,"R$ 288,20000000000005",7,25.40,Aguardando Pgto,Ouro Branco,MG,Brasil,36420-000,10.0,Cartão de Crédito,VICTOR GONÇALVES DONADONI,Lux
143431,0,7/7/2019,11:05:44,PEDRO SILVA RAMOS,Sabonete,"R$ 2,4",15,46.00,Aguardando Pagamento,Ouro Branco,MG,Brasil,36420-000,10.0,Cartão de Crédito,VICTOR GONÇALVES DONADONI,Nivea
4853,0,7/7/2019,11:05:44,PEDRO SILVA RAMOS,Macarrão,"R$ 2,63",14,46.82,Aguardando Pagamento,Ouro Branco,MG,Brasil,3642XXXX,10.0,Cartão de Crédito,VICTOR GONÇALVES DONADONI,Renata
364791,0,7/7/2019,11:05:44,PEDRO SILVA RAMOS,Pasta de Dente,"R$ 2,98",9,36.82,Aguardando Pagamento,Ouro Branco,MG,Brasil,36420-000,10.0,Cartão de Crédito,VICTOR GONÇALVES DONADONI,Colgate
170364,1,8/23/2018,17:52:53,PEDRO DA SILVA BIANQUINI,Sabonete,"R$ 2,37",9,21.33,Pagamento Confirmado,Duque de Caxias,RJ,Brasil,25000-000,0.0,Cartão de Crédito,HENRICO VICTOR ALVES,Dove
102491,1,8/23/2018,17:52:53,PEDRO DA SILVA BIANQUINI,Detergente,"R$ 3,45",15,51.75,Pagamento Confirmado,Duque de Caxias,RJ,Brasil,25000-000,0.0,Cartão de Crédito,HENRICO VICTOR ALVES,Minuano
219754,1,8/23/2018,17:52:53,PEDRO DA SILVA BIANQUINI,Macarrão,"R$ 3,23",8,25.84,Pagamento Confirmado,Duque de Caxias,RJ,Brasil,25000-000,0.0,Cartão de Crédito,HENRICO VICTOR ALVES,Barilla
158811,1,8/23/2018,17:52:53,PEDRO DA SILVA BIANQUINI,Refrigerante,"R$ 6,83",5,34.15,Pagamento Confirmado,Duque de Caxias,RJ,Brasil,25000-000,0.0,Cartão de Crédito,HENRICO VICTOR ALVES,Fanta
40805,2,11/6/2022,7:03:58,LUCAS CUNHA DE OLIVEIRA,Água Mineral,"R$ 1,4",10,23.00,Aguardando Pagamento,Viçosa,MG,Brasil,36570-000,9.0,Transferência Bancária,MICAEL MALAQUIAS DE SOUZA OLIVEIRA,Minalba
86920,2,11/6/2022,7:03:58,LUCAS CUNHA DE OLIVEIRA,Macarrão,"R$ 3,76",6,31.56,Aguardando Pagamento,Viçosa,MG,Brasil,36570-000,9.0,Transferência Bancária,MICAEL MALAQUIAS DE SOUZA OLIVEIRA,Barilla


In [145]:
#Contando quantos valores sao nulos antes de fazer ajustes
display(df['valor'].isnull().sum())

#Removendo o R$ da coluna de valor
df['valor'] = df['valor'].astype(str).str.replace('R$', '', regex=False)

#Removendo os espaços que existem nos valores ou que possam existir
df['valor'] = df['valor'].str.replace(' ', '' ,regex=False)

#Convertendo virgula para ponto
df['valor'] = df['valor'].str.replace(',','.',regex=False)

#Convertendo para float a coluna
df['valor'] = pd.to_numeric(df['valor'], errors='coerce')

#Fazendo a contagem de quantos valores sao nulos apos ajustar 
display(df['valor'].isnull().sum())

#

np.int64(0)

np.int64(83)

In [146]:
display(df.head())

,id_da_compra,data,hora,cliente,produto,valor,quantidade,total,status,cidade,estado,pais,cep,frete,pagamento,vendedor,marca
0,13679,3/20/2021,23:35:51,LUCAS Araujo Kuhn,Queijo Mussarela,16.87,13,239.31,Pagamento Confirmado,Niterói,RJ,Brasil,24000-000,20.0,Cartão de Crédito,SAMUEL HENRIQUE CAÇADOR,Porto Alegre
1,28070,10/30/2020,9:00:53,MICAEL SOUZA RONCETE,Molho de Tomate,3.25,3,9.75,Pagamento Confirmado,Mariana,MG,Brasil,35420-000,0.0,Pix,MICAEL MALAQUIAS DE SOUZA OLIVEIRA,Fugini
2,47484,6/9/2021,15:30:28,FELIPE AUGUSTO NERY SILVA,Água Mineral,1.63,9,36.67,Pagamento Confirmado,Cabo Frio,RJ,Brasil,28900-000,22.0,Transferência Bancária,HENRICO MATOS LIMA DA CUNHA,Minalba
3,20809,6/4/2022,8:41:23,LEVI RIBEIRO AMORIM,Carvão,8.74,4,54.96,Em Separação,Campos dos Goytacazes,RJ,Brasil,28000-000,20.0,Pix,GABRIEL QUEIROZ DE AGUIAR,Marca-Brás
4,47123,5/4/2019,13:38:45,GABRIEL MATOS LIMA DA CUNHA,Café,9.48,2,18.96,Em Separação,Conselheiro Lafaiete,MG,Brasil,36400-000,0.0,Cartão de Crédito,HENRICO MATOS LIMA DA CUNHA,3 Corações


Transformando a coluna valor para numerico

In [147]:
df['valor'] = pd.to_numeric(df['valor'], errors='coerce')
display(df[['valor']].head())

,valor
0,16.87
1,3.25
2,1.63
3,8.74
4,9.48


VERIFICANDO A COLUNA DA DATA

In [148]:
display(df[['data']].head())

,data
0,3/20/2021
1,10/30/2020
2,6/9/2021
3,6/4/2022
4,5/4/2019


Padronizando a coluna de data

In [149]:
df['data'] = pd.to_datetime(df['data'], errors='coerce')
display(df[['data']].head())

,data
0,2021-03-20
1,2020-10-30
2,2021-06-09
3,2022-06-04
4,2019-05-04


In [150]:
display(df[['hora']].head())
display(df[['hora']].tail())

,hora
0,23:35:51
1,9:00:53
2,15:30:28
3,8:41:23
4,13:38:45


,hora
368747,23:00:58
368748,12:46:43
368749,19:09:58
368750,6:59:09
368751,22:47:21


In [151]:
df['hora'] = pd.to_datetime(df['hora'], format='%H:%M:%S').dt.strftime('%H:%M:%S')
display(df[['hora']].head())
display(df[['hora']].tail())


,hora
0,23:35:51
1,09:00:53
2,15:30:28
3,08:41:23
4,13:38:45


,hora
368747,23:00:58
368748,12:46:43
368749,19:09:58
368750,06:59:09
368751,22:47:21


transformando em um padrao as horas para duas casas a cada 2 pontos

In [152]:
display(df.head(10))

,id_da_compra,data,hora,cliente,produto,valor,quantidade,total,status,cidade,estado,pais,cep,frete,pagamento,vendedor,marca
0,13679,2021-03-20,23:35:51,LUCAS Araujo Kuhn,Queijo Mussarela,16.87,13,239.31,Pagamento Confirmado,Niterói,RJ,Brasil,24000-000,20.0,Cartão de Crédito,SAMUEL HENRIQUE CAÇADOR,Porto Alegre
1,28070,2020-10-30,09:00:53,MICAEL SOUZA RONCETE,Molho de Tomate,3.25,3,9.75,Pagamento Confirmado,Mariana,MG,Brasil,35420-000,0.0,Pix,MICAEL MALAQUIAS DE SOUZA OLIVEIRA,Fugini
2,47484,2021-06-09,15:30:28,FELIPE AUGUSTO NERY SILVA,Água Mineral,1.63,9,36.67,Pagamento Confirmado,Cabo Frio,RJ,Brasil,28900-000,22.0,Transferência Bancária,HENRICO MATOS LIMA DA CUNHA,Minalba
3,20809,2022-06-04,08:41:23,LEVI RIBEIRO AMORIM,Carvão,8.74,4,54.96,Em Separação,Campos dos Goytacazes,RJ,Brasil,28000-000,20.0,Pix,GABRIEL QUEIROZ DE AGUIAR,Marca-Brás
4,47123,2019-05-04,13:38:45,GABRIEL MATOS LIMA DA CUNHA,Café,9.48,2,18.96,Em Separação,Conselheiro Lafaiete,MG,Brasil,36400-000,0.0,Cartão de Crédito,HENRICO MATOS LIMA DA CUNHA,3 Corações
5,38623,2018-02-19,17:32:01,LUCAS ANTÔNIO DE SOUZA NETO,Café,10.16,1,20.16,Entregue com Sucesso,Resende Costa,MG,Brasil,36340-000,10.0,Transferência Bancária,VICTOR GONÇALVES DONADONI,3 Corações
6,46627,2018-04-21,22:08:30,GUILHERME IVAIR DE ASSIS SETTA,Feijão,7.77,3,43.31,Em Separação,São Paulo,SP,Brasil,01000-000,20.0,Pix,SAMUEL HENRIQUE CAÇADOR,Caldo Bom
7,53579,2020-09-25,12:04:15,GABRIEL MUZY FERNANDES,Queijo Mussarela,14.93,14,218.02,Em Separação,Bicas,MG,Brasil,36600-000,9.0,Boleto,MICAEL MALAQUIAS DE SOUZA OLIVEIRA,Porto Alegre
8,34924,2018-06-20,17:10:42,LEVI MATOS LIMA DA CUNHA,Condicionador,9.51,5,69.55,Pagamento Confirmado,Cabo Frio,RJ,Brasil,28900-000,22.0,Pix,FELIPE HENRIQUE COSTA BARNABE MARAZO,Pantene
9,10575,2018-10-01,18:36:45,HENRICO DA CUNHA TEIXEIRA,Sabonete,1.25,2,13.00,Em Separação,São João del-Rei,MG,Brasil,36300-000,10.5,Boleto,GABRIEL QUEIROZ DE AGUIAR,Dove


In [153]:
def limpar(texto):
    texto = unicodedata.normalize('NFKD', str(texto)).encode('ASCII', 'ignore').decode('ASCII')
    texto = re.sub(r'[^a-zA-Z0-9 ]', '', texto)
    return texto.lower().strip()
df["produto"] = df["produto"].apply(limpar)
display(df[['produto']].head())



,produto
0,queijo mussarela
1,molho de tomate
2,agua mineral
3,carvao
4,cafe


padronizando os dados da coluna produto removendo acentos e deixnado tudo em minusculo

In [154]:
produtos_unicos = df['produto'].unique()


produtos_lista = produtos_unicos.tolist()

for produto in produtos_lista:
    print(produto)

queijo mussarela
molho de tomate
agua mineral
carvao
cafe
feijao
condicionador
sabonete
acucar
manteiga
papel higienico
oleo de soja
vinho
macarrao
shampoo
presunto
cerveja
papel toalha
pasta de dente
pao de forma
sabao em po
amaciante
desinfetante
refrigerante
arroz
farinha de trigo
biscoito recheado
sal
suco de laranja
leite integral
detergente
refrigkrante
scl
papel qoalha
desinfetanue
qbeijo mussarela
agua mineras
presuntd
macawrao
sucoyde laranja
clfe
desinfekante
majarrao
cnfe
farinha de tripo
manteigt
zabonete
mqcarrao
arroc
tal
sabonepe
papel twalha
agua mineual
condicioiador
condibionador
caft
ieijao
acucaz
amaciayte
caff
mopho de tomate
molmo de tomate
biscoitq recheado
macirrao
deqergente
cafc
queijo mussarelz
deterwente


verificado muitos itens com erros de digitacao

In [155]:
produtos_map ={
    'acucaz': 'acucar',
    'agua mineras': 'agua mineral',
    'agua mineual': 'agua mineral',
    'amaciayte': 'amaciante',
    'arroc': 'arroz',
    'biscoitq recheado': 'biscoito recheado',
    'cafc': 'cafe',
    'caff': 'cafe',
    'caft': 'cafe',
    'clfe': 'cafe',
    'cnfe': 'cafe',
    'condibionador': 'condicionador',
    'condicioiador': 'condicionador',
    'deqergente': 'detergente',
    'deterwente': 'detergente',
    'desinfekante': 'desinfetante',
    'desinfetanue': 'desinfetante',
    'farinha de tripo': 'farinha de trigo',
    'ieijao': 'feijao',
    'macawrao': 'macarrao',
    'macirrao': 'macarrao',
    'majarrao': 'macarrao',
    'mqcarrao': 'macarrao',
    'manteigt': 'manteiga',
    'molmo de tomate': 'molho de tomate',
    'mopho de tomate': 'molho de tomate',
    'papel qoalha': 'papel toalha',
    'papel twalha': 'papel toalha',
    'refrigkrante': 'refrigerante',
    'scl' : 'sal',
    'tal' : 'sal',
    'sabonepe' : 'sabonete',
    'qbeijo mussarela' : 'queijo mussarela',
    'presuntd' : 'presunto',
    'sucoyde laranja' : 'suco de laranja',
    'zabonete' : 'sabonete',
    'queijo mussarelz' : 'queijo mussarela'

}
df["produto"] = df["produto"].replace(produtos_map)
df["produto"] = df["produto"].str.strip()
df["produto"].unique()

array(['queijo mussarela', 'molho de tomate', 'agua mineral', 'carvao',
       'cafe', 'feijao', 'condicionador', 'sabonete', 'acucar',
       'manteiga', 'papel higienico', 'oleo de soja', 'vinho', 'macarrao',
       'shampoo', 'presunto', 'cerveja', 'papel toalha', 'pasta de dente',
       'pao de forma', 'sabao em po', 'amaciante', 'desinfetante',
       'refrigerante', 'arroz', 'farinha de trigo', 'biscoito recheado',
       'sal', 'suco de laranja', 'leite integral', 'detergente'],
      dtype=object)

tirei erros de digtacoes que estavam em alguns itens

In [156]:
display(df.head(10))

,id_da_compra,data,hora,cliente,produto,valor,quantidade,total,status,cidade,estado,pais,cep,frete,pagamento,vendedor,marca
0,13679,2021-03-20,23:35:51,LUCAS Araujo Kuhn,queijo mussarela,16.87,13,239.31,Pagamento Confirmado,Niterói,RJ,Brasil,24000-000,20.0,Cartão de Crédito,SAMUEL HENRIQUE CAÇADOR,Porto Alegre
1,28070,2020-10-30,09:00:53,MICAEL SOUZA RONCETE,molho de tomate,3.25,3,9.75,Pagamento Confirmado,Mariana,MG,Brasil,35420-000,0.0,Pix,MICAEL MALAQUIAS DE SOUZA OLIVEIRA,Fugini
2,47484,2021-06-09,15:30:28,FELIPE AUGUSTO NERY SILVA,agua mineral,1.63,9,36.67,Pagamento Confirmado,Cabo Frio,RJ,Brasil,28900-000,22.0,Transferência Bancária,HENRICO MATOS LIMA DA CUNHA,Minalba
3,20809,2022-06-04,08:41:23,LEVI RIBEIRO AMORIM,carvao,8.74,4,54.96,Em Separação,Campos dos Goytacazes,RJ,Brasil,28000-000,20.0,Pix,GABRIEL QUEIROZ DE AGUIAR,Marca-Brás
4,47123,2019-05-04,13:38:45,GABRIEL MATOS LIMA DA CUNHA,cafe,9.48,2,18.96,Em Separação,Conselheiro Lafaiete,MG,Brasil,36400-000,0.0,Cartão de Crédito,HENRICO MATOS LIMA DA CUNHA,3 Corações
5,38623,2018-02-19,17:32:01,LUCAS ANTÔNIO DE SOUZA NETO,cafe,10.16,1,20.16,Entregue com Sucesso,Resende Costa,MG,Brasil,36340-000,10.0,Transferência Bancária,VICTOR GONÇALVES DONADONI,3 Corações
6,46627,2018-04-21,22:08:30,GUILHERME IVAIR DE ASSIS SETTA,feijao,7.77,3,43.31,Em Separação,São Paulo,SP,Brasil,01000-000,20.0,Pix,SAMUEL HENRIQUE CAÇADOR,Caldo Bom
7,53579,2020-09-25,12:04:15,GABRIEL MUZY FERNANDES,queijo mussarela,14.93,14,218.02,Em Separação,Bicas,MG,Brasil,36600-000,9.0,Boleto,MICAEL MALAQUIAS DE SOUZA OLIVEIRA,Porto Alegre
8,34924,2018-06-20,17:10:42,LEVI MATOS LIMA DA CUNHA,condicionador,9.51,5,69.55,Pagamento Confirmado,Cabo Frio,RJ,Brasil,28900-000,22.0,Pix,FELIPE HENRIQUE COSTA BARNABE MARAZO,Pantene
9,10575,2018-10-01,18:36:45,HENRICO DA CUNHA TEIXEIRA,sabonete,1.25,2,13.00,Em Separação,São João del-Rei,MG,Brasil,36300-000,10.5,Boleto,GABRIEL QUEIROZ DE AGUIAR,Dove


In [157]:
df["produto"] = df["produto"].replace(produtos_map)
df["produto"] = df["produto"].str.strip()
import re

df['produto'] = df['produto'].str.replace(r'[^\w\s]', '', regex=True)
df["produto"].unique()

array(['queijo mussarela', 'molho de tomate', 'agua mineral', 'carvao',
       'cafe', 'feijao', 'condicionador', 'sabonete', 'acucar',
       'manteiga', 'papel higienico', 'oleo de soja', 'vinho', 'macarrao',
       'shampoo', 'presunto', 'cerveja', 'papel toalha', 'pasta de dente',
       'pao de forma', 'sabao em po', 'amaciante', 'desinfetante',
       'refrigerante', 'arroz', 'farinha de trigo', 'biscoito recheado',
       'sal', 'suco de laranja', 'leite integral', 'detergente'],
      dtype=object)

removendo caracteres especiais dos produtos 

In [158]:
display(df[['status']].head(10))

,status
0,Pagamento Confirmado
1,Pagamento Confirmado
2,Pagamento Confirmado
3,Em Separação
4,Em Separação
5,Entregue com Sucesso
6,Em Separação
7,Em Separação
8,Pagamento Confirmado
9,Em Separação


In [159]:
df["status"] = df["status"].apply(limpar)
display(df[['status']].head(10))

,status
0,pagamento confirmado
1,pagamento confirmado
2,pagamento confirmado
3,em separacao
4,em separacao
5,entregue com sucesso
6,em separacao
7,em separacao
8,pagamento confirmado
9,em separacao


padronizando a coluna status tambem caso precise para fazer comparacoes 

In [160]:
display(df.head(10))

,id_da_compra,data,hora,cliente,produto,valor,quantidade,total,status,cidade,estado,pais,cep,frete,pagamento,vendedor,marca
0,13679,2021-03-20,23:35:51,LUCAS Araujo Kuhn,queijo mussarela,16.87,13,239.31,pagamento confirmado,Niterói,RJ,Brasil,24000-000,20.0,Cartão de Crédito,SAMUEL HENRIQUE CAÇADOR,Porto Alegre
1,28070,2020-10-30,09:00:53,MICAEL SOUZA RONCETE,molho de tomate,3.25,3,9.75,pagamento confirmado,Mariana,MG,Brasil,35420-000,0.0,Pix,MICAEL MALAQUIAS DE SOUZA OLIVEIRA,Fugini
2,47484,2021-06-09,15:30:28,FELIPE AUGUSTO NERY SILVA,agua mineral,1.63,9,36.67,pagamento confirmado,Cabo Frio,RJ,Brasil,28900-000,22.0,Transferência Bancária,HENRICO MATOS LIMA DA CUNHA,Minalba
3,20809,2022-06-04,08:41:23,LEVI RIBEIRO AMORIM,carvao,8.74,4,54.96,em separacao,Campos dos Goytacazes,RJ,Brasil,28000-000,20.0,Pix,GABRIEL QUEIROZ DE AGUIAR,Marca-Brás
4,47123,2019-05-04,13:38:45,GABRIEL MATOS LIMA DA CUNHA,cafe,9.48,2,18.96,em separacao,Conselheiro Lafaiete,MG,Brasil,36400-000,0.0,Cartão de Crédito,HENRICO MATOS LIMA DA CUNHA,3 Corações
5,38623,2018-02-19,17:32:01,LUCAS ANTÔNIO DE SOUZA NETO,cafe,10.16,1,20.16,entregue com sucesso,Resende Costa,MG,Brasil,36340-000,10.0,Transferência Bancária,VICTOR GONÇALVES DONADONI,3 Corações
6,46627,2018-04-21,22:08:30,GUILHERME IVAIR DE ASSIS SETTA,feijao,7.77,3,43.31,em separacao,São Paulo,SP,Brasil,01000-000,20.0,Pix,SAMUEL HENRIQUE CAÇADOR,Caldo Bom
7,53579,2020-09-25,12:04:15,GABRIEL MUZY FERNANDES,queijo mussarela,14.93,14,218.02,em separacao,Bicas,MG,Brasil,36600-000,9.0,Boleto,MICAEL MALAQUIAS DE SOUZA OLIVEIRA,Porto Alegre
8,34924,2018-06-20,17:10:42,LEVI MATOS LIMA DA CUNHA,condicionador,9.51,5,69.55,pagamento confirmado,Cabo Frio,RJ,Brasil,28900-000,22.0,Pix,FELIPE HENRIQUE COSTA BARNABE MARAZO,Pantene
9,10575,2018-10-01,18:36:45,HENRICO DA CUNHA TEIXEIRA,sabonete,1.25,2,13.00,em separacao,São João del-Rei,MG,Brasil,36300-000,10.5,Boleto,GABRIEL QUEIROZ DE AGUIAR,Dove


In [161]:
display(df.isnull().sum())
df[df["valor"].isnull()]


id_da_compra       0
data               0
hora               0
cliente            0
produto            0
valor             83
quantidade         0
total           3685
status             0
cidade             0
estado             0
pais               0
cep                0
frete           7371
pagamento          0
vendedor        3680
marca              0
dtype: int64

,id_da_compra,data,hora,cliente,produto,valor,quantidade,total,status,cidade,estado,pais,cep,frete,pagamento,vendedor,marca
394,29431,2019-01-12,14:02:00,THIAGO QUEIROZ DE AGUIAR,presunto,NaN,15,184.15,transportando,Volta Redonda,RJ,Brasil,27200-000,19.0,Transferência Bancária,FELIPE HENRIQUE COSTA BARNABE MARAZO,Aurora
7718,53850,2022-08-26,20:18:56,JOÃO QUEIROZ DE AGUIAR,pasta de dente,NaN,7,31.91,pagamento confirmado,Belo Horizonte,MG,Brasil,30000-000,10.0,Cartão de Crédito,HENRICO MATOS LIMA DA CUNHA,Sorriso
12960,47174,2022-10-26,17:09:55,LEVI ANTÔNIO DE SOUZA NETO,molho de tomate,NaN,10,35.40,entregue,São Vicente de Minas,MG,Brasil,37370-000,0.0,Transferência Bancária,PAULO SOUZA RONCETE,Elefante
15553,17975,2021-11-15,21:23:55,SAMUEL MATHEUS DA SILVA,papel toalha,NaN,10,34.70,entregue,Tiradentes,MG,Brasil,36325-000,10.0,Cartão de Crédito,HENRICO MATOS LIMA DA CUNHA,Elite
15575,41147,2019-09-12,16:30:35,MICAEL CARDOSO DINIZ,sal,NaN,4,8.64,em transporte,Santo André,SP,Brasil,09000-000,0.0,Transferência Bancária,HENRICO VICTOR ALVES,Cisne
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
355903,26209,2018-08-17,14:41:11,KALLEBE HENRIQUE CAÇADOR,macarrao,NaN,9,28.63,em separacao,Astolfo Dutra,MG,Brasil,36780-000,10.0,Pix,HENRICO VICTOR ALVES,Renata
355949,16758,2018-03-25,02:34:22,VICTOR SANTOS DA SILVA,amaciante,NaN,15,110.65,entregue,Resende Costa,MG,Brasil,36340-000,-10.0,Cartão de Crédito,GABRIEL QUEIROZ DE AGUIAR,Fofo
361106,30157,2018-04-28,12:44:13,GABRIEL JOSÉ SILVA MATIAS,papel higienico,NaN,15,62.10,pc,Duque de Caxias,RJ,Brasil,25000-000,0.0,Pix,FELIPE HENRIQUE COSTA BARNABE MARAZO,Personal
362586,52987,2020-11-19,22:24:50,LUCAS QUEIROZ DE AGUIAR,sal,NaN,2,14.24,em separacao,São Tiago,MG,Brasil,36350-000,9.5,Transferência Bancária,FELIPE HENRIQUE COSTA BARNABE MARAZO,Lebre


Verificando quantos valores sao nulos

In [162]:
mediana_por_produto = df.groupby("produto")["valor"].median()
df["valor"] = df.apply(
    lambda row: mediana_por_produto[row["produto"]] if pd.isnull(row["valor"]) else row["valor"],
    axis=1
)
display(df.isnull().sum())
df[df["valor"].isnull()]

id_da_compra       0
data               0
hora               0
cliente            0
produto            0
valor              0
quantidade         0
total           3685
status             0
cidade             0
estado             0
pais               0
cep                0
frete           7371
pagamento          0
vendedor        3680
marca              0
dtype: int64

,id_da_compra,data,hora,cliente,produto,valor,quantidade,total,status,cidade,estado,pais,cep,frete,pagamento,vendedor,marca


verifiquei que existiam itens com valores NAN fiz uma mediana com os produtos que sao iguais e inseri nos valores onde estavam NAN 



In [163]:
display(df.head(10))

,id_da_compra,data,hora,cliente,produto,valor,quantidade,total,status,cidade,estado,pais,cep,frete,pagamento,vendedor,marca
0,13679,2021-03-20,23:35:51,LUCAS Araujo Kuhn,queijo mussarela,16.87,13,239.31,pagamento confirmado,Niterói,RJ,Brasil,24000-000,20.0,Cartão de Crédito,SAMUEL HENRIQUE CAÇADOR,Porto Alegre
1,28070,2020-10-30,09:00:53,MICAEL SOUZA RONCETE,molho de tomate,3.25,3,9.75,pagamento confirmado,Mariana,MG,Brasil,35420-000,0.0,Pix,MICAEL MALAQUIAS DE SOUZA OLIVEIRA,Fugini
2,47484,2021-06-09,15:30:28,FELIPE AUGUSTO NERY SILVA,agua mineral,1.63,9,36.67,pagamento confirmado,Cabo Frio,RJ,Brasil,28900-000,22.0,Transferência Bancária,HENRICO MATOS LIMA DA CUNHA,Minalba
3,20809,2022-06-04,08:41:23,LEVI RIBEIRO AMORIM,carvao,8.74,4,54.96,em separacao,Campos dos Goytacazes,RJ,Brasil,28000-000,20.0,Pix,GABRIEL QUEIROZ DE AGUIAR,Marca-Brás
4,47123,2019-05-04,13:38:45,GABRIEL MATOS LIMA DA CUNHA,cafe,9.48,2,18.96,em separacao,Conselheiro Lafaiete,MG,Brasil,36400-000,0.0,Cartão de Crédito,HENRICO MATOS LIMA DA CUNHA,3 Corações
5,38623,2018-02-19,17:32:01,LUCAS ANTÔNIO DE SOUZA NETO,cafe,10.16,1,20.16,entregue com sucesso,Resende Costa,MG,Brasil,36340-000,10.0,Transferência Bancária,VICTOR GONÇALVES DONADONI,3 Corações
6,46627,2018-04-21,22:08:30,GUILHERME IVAIR DE ASSIS SETTA,feijao,7.77,3,43.31,em separacao,São Paulo,SP,Brasil,01000-000,20.0,Pix,SAMUEL HENRIQUE CAÇADOR,Caldo Bom
7,53579,2020-09-25,12:04:15,GABRIEL MUZY FERNANDES,queijo mussarela,14.93,14,218.02,em separacao,Bicas,MG,Brasil,36600-000,9.0,Boleto,MICAEL MALAQUIAS DE SOUZA OLIVEIRA,Porto Alegre
8,34924,2018-06-20,17:10:42,LEVI MATOS LIMA DA CUNHA,condicionador,9.51,5,69.55,pagamento confirmado,Cabo Frio,RJ,Brasil,28900-000,22.0,Pix,FELIPE HENRIQUE COSTA BARNABE MARAZO,Pantene
9,10575,2018-10-01,18:36:45,HENRICO DA CUNHA TEIXEIRA,sabonete,1.25,2,13.00,em separacao,São João del-Rei,MG,Brasil,36300-000,10.5,Boleto,GABRIEL QUEIROZ DE AGUIAR,Dove


In [164]:

df[df["total"].isnull()]

,id_da_compra,data,hora,cliente,produto,valor,quantidade,total,status,cidade,estado,pais,cep,frete,pagamento,vendedor,marca
320,15092,2018-03-30,21:29:12,GUILHERME HUGO CAZULA PEREIRA,feijao,10.19,2,NaN,em separacao,Itaboraí,RJ,Brasil,24800-000,0.0,Cartão de Crédito,GABRIEL QUEIROZ DE AGUIAR,Caldo Bom
410,33369,2020-08-21,09:10:58,GABRIEL PAULO ALVES ALVIM,refrigerante,6.21,2,NaN,em transporte,Angra dos Reis,RJ,Brasil,23900-000,0.0,Transferência Bancária,HENRICO MATOS LIMA DA CUNHA,Coca-Cola
448,28894,2022-01-09,15:55:37,THIAGO HENRIQUE CAÇADOR,macarrao,2.49,14,NaN,em transporte,Cataguases,MG,Brasil,36770-000,0.0,Pix,LUCAS VITOR FAÇANHA NEVES,Renata
561,15983,2022-10-26,17:31:51,LUCAS RISSON EUZEBIO,cafe,8.01,6,NaN,em separacao,Astolfo Dutra,MG,Brasil,36780-000,0.0,Transferência Bancária,VICTOR GONÇALVES DONADONI,3 Corações
565,38996,2022-10-15,14:36:22,THIAGO CARDOSO DINIZ,cafe,12.13,2,NaN,pagamento confirmado,São Bernardo do Campo,SP,Brasil,09700-000,0.0,Cartão de Crédito,MICAEL MALAQUIAS DE SOUZA OLIVEIRA,Melitta
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
368161,31960,2020-04-16,16:55:10,CARLOS ANTÔNIO DE DECCO OLIVEIRA,vinho,25.21,1,NaN,em transporte,Conselheiro Lafaiete,MG,Brasil,36400-000,10.5,Pix,GABRIEL QUEIROZ DE AGUIAR,Concha y Toro
368302,49778,2018-09-03,22:12:22,LETICIA GONÇALVES DONADONI,suco de laranja,7.27,14,NaN,aguardando pagamento,São João Nepomuceno,MG,Brasil,36680-000,12.0,Transferência Bancária,CARLOS QUEIROZ DE AGUIAR,Dafruta
368334,30561,2018-04-22,12:44:13,SAMUEL MUZY FERNANDES,suco de laranja,8.18,7,NaN,pagamento confirmado,Duque de Caxias,RJ,Brasil,25000-000,19.0,Pix,MICAEL MALAQUIAS DE SOUZA OLIVEIRA,Dafruta
368562,53991,2018-07-25,10:39:59,LETICIA MATOS LIMA DA CUNHA,cafe,14.34,1,NaN,aguardando pagamento,Santo André,SP,Brasil,09000-000,18.0,Pix,SAMUEL HENRIQUE CAÇADOR,Melitta


MOSTRANDO VALORES TOTAIS IDENTIFICADOS COM NULOS

In [165]:
display(df[df['frete'] < 0])

,id_da_compra,data,hora,cliente,produto,valor,quantidade,total,status,cidade,estado,pais,cep,frete,pagamento,vendedor,marca
71,17370,2022-03-01,19:06:27,JOÃO QUEIROZ DE AGUIAR,queijo mussarela,16.51,14,241.14,sep,Resende Costa,MG,Brasil,36340-000,-10.0,Pix,HENRICO MATOS LIMA DA CUNHA,Tirolez
99,21247,2022-01-20,16:06:33,MARCO SANTOS DA SILVA,macarrao,454.57,13,55.11,aguardando pagamento,Lagoa Dourada,MG,Brasil,36345-000,-100.0,Cartão de Crédito,MICAEL MALAQUIAS DE SOUZA OLIVEIRA,Adria
215,9551,2020-04-26,18:01:27,JOÃO DA CUNHA TEIXEIRA,manteiga,8.37,12,118.44,em separacao,São João de Meriti,RJ,Brasil,25500-000,-18.0,Cartão de Crédito,VICTOR GONÇALVES DONADONI,Qualy
382,8575,2022-04-23,12:01:48,YURI MUZY FERNANDES,agua mineral,2.02,13,36.26,transp,Tiradentes,MG,Brasil,36325-000,-10.0,Cartão de Crédito,HENRICO MATOS LIMA DA CUNHA,São Lourenço
451,19426,2020-04-23,19:51:29,JOÃO QUEIROZ DE AGUIAR,sabonete,1.39,6,16.84,aguardando pagamento,Tocantins,MG,Brasil,36510-000,-8.5,Transferência Bancária,GABRIEL QUEIROZ DE AGUIAR,Lux
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
368491,24485,2021-02-16,03:32:56,KALLEBE SEGURASSE GONÇALVES,acucar,2.92,2,15.84,aguardando pagamento,Belo Horizonte,MG,Brasil,30000-000,-10.0,Pix,CARLOS QUEIROZ DE AGUIAR,Caravelas
368594,16893,2019-03-05,03:17:24,BLENDOW CASTILHO CARDOSO,amaciante,4.78,15,81.70,aguardando pagamento,Lima Duarte,MG,Brasil,36140-000,-10.0,Cartão de Crédito,HENRICO MATOS LIMA DA CUNHA,Downy
368635,43042,2020-10-25,10:58:55,JOÃO LACERDA PORTUGAL DA SILVA,suco de laranja,7.44,11,98.84,pagamento confirmado,Campinas,SP,Brasil,13000-000,-17.0,Transferência Bancária,MICAEL MALAQUIAS DE SOUZA OLIVEIRA,Del Valle
368684,35303,2021-03-13,09:18:57,PEDRO DA SILVA BIANQUINI,farinha de trigo,2.35,7,27.45,em transporte,Ubá,MG,Brasil,36500-000,-11.0,Cartão de Crédito,PAULO SOUZA RONCETE,Anaconda


verificando quais fretes sao negativos   

In [166]:
mediana_frete_por_cidade = df[df['frete'] > 0].groupby('cidade')['frete'].median().to_dict()


df['frete'] = df.apply(
    lambda row: row['frete'] if row['frete'] >= 0 else mediana_frete_por_cidade.get(row['cidade'], row['frete']),
    axis=1
)

Mantém o valor atual do frete se for maior ou igual a 0.
Substitui pelo valor mediano da cidade apenas se o frete for negativo (< 0).

In [167]:
mediana_frete_por_cidade = df[df['frete'] > 0].groupby('cidade')['frete'].median().to_dict()

df['frete'] = df.apply(
    lambda row: mediana_frete_por_cidade.get(row['cidade'], row['frete']) if row['frete'] == 0 else row['frete'],
    axis=1
)

Substitui somente os fretes com valor 0 pela mediana da cidade.
Mantém todos os demais valores, inclusive negativos.

In [168]:
display(df.isnull().sum())
display(df[df['frete'] < 0])

id_da_compra       0
data               0
hora               0
cliente            0
produto            0
valor              0
quantidade         0
total           3685
status             0
cidade             0
estado             0
pais               0
cep                0
frete              0
pagamento          0
vendedor        3680
marca              0
dtype: int64

,id_da_compra,data,hora,cliente,produto,valor,quantidade,total,status,cidade,estado,pais,cep,frete,pagamento,vendedor,marca


verificando se nao existem mais valores nulos para o frete

In [169]:
display(df.head(10))

,id_da_compra,data,hora,cliente,produto,valor,quantidade,total,status,cidade,estado,pais,cep,frete,pagamento,vendedor,marca
0,13679,2021-03-20,23:35:51,LUCAS Araujo Kuhn,queijo mussarela,16.87,13,239.31,pagamento confirmado,Niterói,RJ,Brasil,24000-000,20.0,Cartão de Crédito,SAMUEL HENRIQUE CAÇADOR,Porto Alegre
1,28070,2020-10-30,09:00:53,MICAEL SOUZA RONCETE,molho de tomate,3.25,3,9.75,pagamento confirmado,Mariana,MG,Brasil,35420-000,12.0,Pix,MICAEL MALAQUIAS DE SOUZA OLIVEIRA,Fugini
2,47484,2021-06-09,15:30:28,FELIPE AUGUSTO NERY SILVA,agua mineral,1.63,9,36.67,pagamento confirmado,Cabo Frio,RJ,Brasil,28900-000,22.0,Transferência Bancária,HENRICO MATOS LIMA DA CUNHA,Minalba
3,20809,2022-06-04,08:41:23,LEVI RIBEIRO AMORIM,carvao,8.74,4,54.96,em separacao,Campos dos Goytacazes,RJ,Brasil,28000-000,20.0,Pix,GABRIEL QUEIROZ DE AGUIAR,Marca-Brás
4,47123,2019-05-04,13:38:45,GABRIEL MATOS LIMA DA CUNHA,cafe,9.48,2,18.96,em separacao,Conselheiro Lafaiete,MG,Brasil,36400-000,10.5,Cartão de Crédito,HENRICO MATOS LIMA DA CUNHA,3 Corações
5,38623,2018-02-19,17:32:01,LUCAS ANTÔNIO DE SOUZA NETO,cafe,10.16,1,20.16,entregue com sucesso,Resende Costa,MG,Brasil,36340-000,10.0,Transferência Bancária,VICTOR GONÇALVES DONADONI,3 Corações
6,46627,2018-04-21,22:08:30,GUILHERME IVAIR DE ASSIS SETTA,feijao,7.77,3,43.31,em separacao,São Paulo,SP,Brasil,01000-000,20.0,Pix,SAMUEL HENRIQUE CAÇADOR,Caldo Bom
7,53579,2020-09-25,12:04:15,GABRIEL MUZY FERNANDES,queijo mussarela,14.93,14,218.02,em separacao,Bicas,MG,Brasil,36600-000,9.0,Boleto,MICAEL MALAQUIAS DE SOUZA OLIVEIRA,Porto Alegre
8,34924,2018-06-20,17:10:42,LEVI MATOS LIMA DA CUNHA,condicionador,9.51,5,69.55,pagamento confirmado,Cabo Frio,RJ,Brasil,28900-000,22.0,Pix,FELIPE HENRIQUE COSTA BARNABE MARAZO,Pantene
9,10575,2018-10-01,18:36:45,HENRICO DA CUNHA TEIXEIRA,sabonete,1.25,2,13.00,em separacao,São João del-Rei,MG,Brasil,36300-000,10.5,Boleto,GABRIEL QUEIROZ DE AGUIAR,Dove


In [170]:
def remover_acentos(texto):
    if pd.isnull(texto):
        return texto
    return unicodedata.normalize('NFKD', texto).encode('ASCII', 'ignore').decode('utf-8')

df['pagamento'] = df['pagamento'].apply(remover_acentos)
df['pagamento'] = df['pagamento'].str.upper()
display(df[['pagamento']].head(10))

,pagamento
0,CARTAO DE CREDITO
1,PIX
2,TRANSFERENCIA BANCARIA
3,PIX
4,CARTAO DE CREDITO
5,TRANSFERENCIA BANCARIA
6,PIX
7,BOLETO
8,PIX
9,BOLETO


estou padronizando a forma de pagamento tirando acentos e caracteres especiais e deixando tudo em maiusculo

In [171]:
display(df.isnull().sum())
display(df.head(10))

id_da_compra       0
data               0
hora               0
cliente            0
produto            0
valor              0
quantidade         0
total           3685
status             0
cidade             0
estado             0
pais               0
cep                0
frete              0
pagamento          0
vendedor        3680
marca              0
dtype: int64

,id_da_compra,data,hora,cliente,produto,valor,quantidade,total,status,cidade,estado,pais,cep,frete,pagamento,vendedor,marca
0,13679,2021-03-20,23:35:51,LUCAS Araujo Kuhn,queijo mussarela,16.87,13,239.31,pagamento confirmado,Niterói,RJ,Brasil,24000-000,20.0,CARTAO DE CREDITO,SAMUEL HENRIQUE CAÇADOR,Porto Alegre
1,28070,2020-10-30,09:00:53,MICAEL SOUZA RONCETE,molho de tomate,3.25,3,9.75,pagamento confirmado,Mariana,MG,Brasil,35420-000,12.0,PIX,MICAEL MALAQUIAS DE SOUZA OLIVEIRA,Fugini
2,47484,2021-06-09,15:30:28,FELIPE AUGUSTO NERY SILVA,agua mineral,1.63,9,36.67,pagamento confirmado,Cabo Frio,RJ,Brasil,28900-000,22.0,TRANSFERENCIA BANCARIA,HENRICO MATOS LIMA DA CUNHA,Minalba
3,20809,2022-06-04,08:41:23,LEVI RIBEIRO AMORIM,carvao,8.74,4,54.96,em separacao,Campos dos Goytacazes,RJ,Brasil,28000-000,20.0,PIX,GABRIEL QUEIROZ DE AGUIAR,Marca-Brás
4,47123,2019-05-04,13:38:45,GABRIEL MATOS LIMA DA CUNHA,cafe,9.48,2,18.96,em separacao,Conselheiro Lafaiete,MG,Brasil,36400-000,10.5,CARTAO DE CREDITO,HENRICO MATOS LIMA DA CUNHA,3 Corações
5,38623,2018-02-19,17:32:01,LUCAS ANTÔNIO DE SOUZA NETO,cafe,10.16,1,20.16,entregue com sucesso,Resende Costa,MG,Brasil,36340-000,10.0,TRANSFERENCIA BANCARIA,VICTOR GONÇALVES DONADONI,3 Corações
6,46627,2018-04-21,22:08:30,GUILHERME IVAIR DE ASSIS SETTA,feijao,7.77,3,43.31,em separacao,São Paulo,SP,Brasil,01000-000,20.0,PIX,SAMUEL HENRIQUE CAÇADOR,Caldo Bom
7,53579,2020-09-25,12:04:15,GABRIEL MUZY FERNANDES,queijo mussarela,14.93,14,218.02,em separacao,Bicas,MG,Brasil,36600-000,9.0,BOLETO,MICAEL MALAQUIAS DE SOUZA OLIVEIRA,Porto Alegre
8,34924,2018-06-20,17:10:42,LEVI MATOS LIMA DA CUNHA,condicionador,9.51,5,69.55,pagamento confirmado,Cabo Frio,RJ,Brasil,28900-000,22.0,PIX,FELIPE HENRIQUE COSTA BARNABE MARAZO,Pantene
9,10575,2018-10-01,18:36:45,HENRICO DA CUNHA TEIXEIRA,sabonete,1.25,2,13.00,em separacao,São João del-Rei,MG,Brasil,36300-000,10.5,BOLETO,GABRIEL QUEIROZ DE AGUIAR,Dove


In [172]:
display(df[['marca']].head(10))

,marca
0,Porto Alegre
1,Fugini
2,Minalba
3,Marca-Brás
4,3 Corações
5,3 Corações
6,Caldo Bom
7,Porto Alegre
8,Pantene
9,Dove


In [173]:
df['cliente'] = df['cliente'].str.upper()
df['vendedor'] = df['vendedor'].str.upper()
display(df[['cliente','vendedor']].head(10))

,cliente,vendedor
0,LUCAS ARAUJO KUHN,SAMUEL HENRIQUE CAÇADOR
1,MICAEL SOUZA RONCETE,MICAEL MALAQUIAS DE SOUZA OLIVEIRA
2,FELIPE AUGUSTO NERY SILVA,HENRICO MATOS LIMA DA CUNHA
3,LEVI RIBEIRO AMORIM,GABRIEL QUEIROZ DE AGUIAR
4,GABRIEL MATOS LIMA DA CUNHA,HENRICO MATOS LIMA DA CUNHA
5,LUCAS ANTÔNIO DE SOUZA NETO,VICTOR GONÇALVES DONADONI
6,GUILHERME IVAIR DE ASSIS SETTA,SAMUEL HENRIQUE CAÇADOR
7,GABRIEL MUZY FERNANDES,MICAEL MALAQUIAS DE SOUZA OLIVEIRA
8,LEVI MATOS LIMA DA CUNHA,FELIPE HENRIQUE COSTA BARNABE MARAZO
9,HENRICO DA CUNHA TEIXEIRA,GABRIEL QUEIROZ DE AGUIAR


transformando tdos os dados dos clientes e vendedor para maisculo padronizando 

In [174]:
duplicadas_vendedores = df.groupby('id_da_compra')['vendedor'].nunique().reset_index()
compras_vendedores = duplicadas_vendedores[duplicadas_vendedores['vendedor'] > 1]

print(compras_vendedores)
df['vendedor'] = df.groupby('id_da_compra')['vendedor'].transform(lambda x: x.ffill().bfill())

Empty DataFrame
Columns: [id_da_compra, vendedor]
Index: []


C:\Users\geize\AppData\Local\Temp\ipykernel_25376\2807877592.py:5: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['vendedor'] = df.groupby('id_da_compra')['vendedor'].transform(lambda x: x.ffill().bfill())


Esse código identifica compras com mais de um vendedor e, depois, preenche os valores faltantes ou errados na coluna vendedor, usando os dados corretos da própria compra. Assim, todas as linhas da mesma compra ficam com o mesmo vendedor.

In [175]:
df_vendedor_null = df[df['vendedor'].isna()]
display(df_vendedor_null)

,id_da_compra,data,hora,cliente,produto,valor,quantidade,total,status,cidade,estado,pais,cep,frete,pagamento,vendedor,marca
99695,13035,2022-05-22,14:54:49,JOÃO DA CUNHA TEIXEIRA,desinfetante,5.31,11,58.41,pagamento confirmado,Cabo Frio,RJ,Brasil,28900-000,22.0,CARTAO DE CREDITO,NaN,Lysoform
156700,13035,2022-05-22,14:54:49,JOÃO DA CUNHA TEIXEIRA,feijao,7.93,3,NaN,pagamento confirmado,Cabo Frio,RJ,Brasil,28900-000,22.0,CARTAO DE CREDITO,NaN,Kicaldo


verificando quis vendedores sao nulos 

In [176]:
df = df.dropna(subset=['vendedor'])
df['vendedor'].isna().sum()

np.int64(0)

deletando vendedores nulos 

In [177]:
df.isnull().sum()

id_da_compra       0
data               0
hora               0
cliente            0
produto            0
valor              0
quantidade         0
total           3684
status             0
cidade             0
estado             0
pais               0
cep                0
frete              0
pagamento          0
vendedor           0
marca              0
dtype: int64

In [178]:
display(df[['total']].head(10))

,total
0,239.31
1,9.75
2,36.67
3,54.96
4,18.96
5,20.16
6,43.31
7,218.02
8,69.55
9,13.00


In [179]:
df[df['total'].isnull()]


,id_da_compra,data,hora,cliente,produto,valor,quantidade,total,status,cidade,estado,pais,cep,frete,pagamento,vendedor,marca
320,15092,2018-03-30,21:29:12,GUILHERME HUGO CAZULA PEREIRA,feijao,10.19,2,NaN,em separacao,Itaboraí,RJ,Brasil,24800-000,18.5,CARTAO DE CREDITO,GABRIEL QUEIROZ DE AGUIAR,Caldo Bom
410,33369,2020-08-21,09:10:58,GABRIEL PAULO ALVES ALVIM,refrigerante,6.21,2,NaN,em transporte,Angra dos Reis,RJ,Brasil,23900-000,23.0,TRANSFERENCIA BANCARIA,HENRICO MATOS LIMA DA CUNHA,Coca-Cola
448,28894,2022-01-09,15:55:37,THIAGO HENRIQUE CAÇADOR,macarrao,2.49,14,NaN,em transporte,Cataguases,MG,Brasil,36770-000,10.5,PIX,LUCAS VITOR FAÇANHA NEVES,Renata
561,15983,2022-10-26,17:31:51,LUCAS RISSON EUZEBIO,cafe,8.01,6,NaN,em separacao,Astolfo Dutra,MG,Brasil,36780-000,10.0,TRANSFERENCIA BANCARIA,VICTOR GONÇALVES DONADONI,3 Corações
565,38996,2022-10-15,14:36:22,THIAGO CARDOSO DINIZ,cafe,12.13,2,NaN,pagamento confirmado,São Bernardo do Campo,SP,Brasil,09700-000,18.0,CARTAO DE CREDITO,MICAEL MALAQUIAS DE SOUZA OLIVEIRA,Melitta
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
368161,31960,2020-04-16,16:55:10,CARLOS ANTÔNIO DE DECCO OLIVEIRA,vinho,25.21,1,NaN,em transporte,Conselheiro Lafaiete,MG,Brasil,36400-000,10.5,PIX,GABRIEL QUEIROZ DE AGUIAR,Concha y Toro
368302,49778,2018-09-03,22:12:22,LETICIA GONÇALVES DONADONI,suco de laranja,7.27,14,NaN,aguardando pagamento,São João Nepomuceno,MG,Brasil,36680-000,12.0,TRANSFERENCIA BANCARIA,CARLOS QUEIROZ DE AGUIAR,Dafruta
368334,30561,2018-04-22,12:44:13,SAMUEL MUZY FERNANDES,suco de laranja,8.18,7,NaN,pagamento confirmado,Duque de Caxias,RJ,Brasil,25000-000,19.0,PIX,MICAEL MALAQUIAS DE SOUZA OLIVEIRA,Dafruta
368562,53991,2018-07-25,10:39:59,LETICIA MATOS LIMA DA CUNHA,cafe,14.34,1,NaN,aguardando pagamento,Santo André,SP,Brasil,09000-000,18.0,PIX,SAMUEL HENRIQUE CAÇADOR,Melitta


In [180]:
df['total'] = df['total'].astype(str).str.replace(',', '.')
df['total'] = df['total'].astype(float).round(2)

Esse código transforma a coluna total em número decimal (float), trocando vírgula por ponto e arredondando para duas casas decimais. Isso garante que os valores estejam no formato certo para cálculos.

In [181]:
display(df.isnull().sum())

id_da_compra       0
data               0
hora               0
cliente            0
produto            0
valor              0
quantidade         0
total           3684
status             0
cidade             0
estado             0
pais               0
cep                0
frete              0
pagamento          0
vendedor           0
marca              0
dtype: int64

In [182]:
media_por_produto_marca = df.groupby(['produto', 'marca'])['valor'].mean().reset_index()

for _, valor in media_por_produto_marca.iterrows():
    print(f"Produto: {valor['produto']}, Marca: {valor['marca']}, Média do Valor: {valor['valor']}")

Produto: acucar, Marca: Caravelas, Média do Valor: 10.171457846238482
Produto: acucar, Marca: Itamarati, Média do Valor: 11.444061962134251
Produto: acucar, Marca: União, Média do Valor: 10.633498498498497
Produto: agua mineral, Marca: Crystal, Média do Valor: 6.906218112244899
Produto: agua mineral, Marca: Minalba, Média do Valor: 7.273298267326733
Produto: agua mineral, Marca: São Lourenço, Média do Valor: 7.0899629743906205
Produto: amaciante, Marca: Confort, Média do Valor: 20.01898395721925
Produto: amaciante, Marca: Downy, Média do Valor: 22.06915764139591
Produto: amaciante, Marca: Fofo, Média do Valor: 19.74499692685925
Produto: arroz, Marca: Camil, Média do Valor: 15.89740775780511
Produto: arroz, Marca: Tio João, Média do Valor: 18.005830540928674
Produto: arroz, Marca: Urbano, Média do Valor: 17.07263005780347
Produto: biscoito recheado, Marca: Bauducco, Média do Valor: 10.76255174544331
Produto: biscoito recheado, Marca: Nestlé, Média do Valor: 10.49132196847861
Produto: bi

Esse código calcula a média de valor de cada produto por marca e depois imprime o resultado mostrando produto, marca e o valor médio. Ele ajuda a entender quais marcas são mais caras ou mais baratas dentro de cada tipo de produto.

In [183]:
display(df.head(10))

,id_da_compra,data,hora,cliente,produto,valor,quantidade,total,status,cidade,estado,pais,cep,frete,pagamento,vendedor,marca
0,13679,2021-03-20,23:35:51,LUCAS ARAUJO KUHN,queijo mussarela,16.87,13,239.31,pagamento confirmado,Niterói,RJ,Brasil,24000-000,20.0,CARTAO DE CREDITO,SAMUEL HENRIQUE CAÇADOR,Porto Alegre
1,28070,2020-10-30,09:00:53,MICAEL SOUZA RONCETE,molho de tomate,3.25,3,9.75,pagamento confirmado,Mariana,MG,Brasil,35420-000,12.0,PIX,MICAEL MALAQUIAS DE SOUZA OLIVEIRA,Fugini
2,47484,2021-06-09,15:30:28,FELIPE AUGUSTO NERY SILVA,agua mineral,1.63,9,36.67,pagamento confirmado,Cabo Frio,RJ,Brasil,28900-000,22.0,TRANSFERENCIA BANCARIA,HENRICO MATOS LIMA DA CUNHA,Minalba
3,20809,2022-06-04,08:41:23,LEVI RIBEIRO AMORIM,carvao,8.74,4,54.96,em separacao,Campos dos Goytacazes,RJ,Brasil,28000-000,20.0,PIX,GABRIEL QUEIROZ DE AGUIAR,Marca-Brás
4,47123,2019-05-04,13:38:45,GABRIEL MATOS LIMA DA CUNHA,cafe,9.48,2,18.96,em separacao,Conselheiro Lafaiete,MG,Brasil,36400-000,10.5,CARTAO DE CREDITO,HENRICO MATOS LIMA DA CUNHA,3 Corações
5,38623,2018-02-19,17:32:01,LUCAS ANTÔNIO DE SOUZA NETO,cafe,10.16,1,20.16,entregue com sucesso,Resende Costa,MG,Brasil,36340-000,10.0,TRANSFERENCIA BANCARIA,VICTOR GONÇALVES DONADONI,3 Corações
6,46627,2018-04-21,22:08:30,GUILHERME IVAIR DE ASSIS SETTA,feijao,7.77,3,43.31,em separacao,São Paulo,SP,Brasil,01000-000,20.0,PIX,SAMUEL HENRIQUE CAÇADOR,Caldo Bom
7,53579,2020-09-25,12:04:15,GABRIEL MUZY FERNANDES,queijo mussarela,14.93,14,218.02,em separacao,Bicas,MG,Brasil,36600-000,9.0,BOLETO,MICAEL MALAQUIAS DE SOUZA OLIVEIRA,Porto Alegre
8,34924,2018-06-20,17:10:42,LEVI MATOS LIMA DA CUNHA,condicionador,9.51,5,69.55,pagamento confirmado,Cabo Frio,RJ,Brasil,28900-000,22.0,PIX,FELIPE HENRIQUE COSTA BARNABE MARAZO,Pantene
9,10575,2018-10-01,18:36:45,HENRICO DA CUNHA TEIXEIRA,sabonete,1.25,2,13.00,em separacao,São João del-Rei,MG,Brasil,36300-000,10.5,BOLETO,GABRIEL QUEIROZ DE AGUIAR,Dove


In [184]:

media_desvio = df.groupby(['produto', 'marca'])['valor'].agg(['mean', 'std'])

df = df.merge(media_desvio, on=['produto', 'marca'], how='left')

df['outlier'] = (df['valor'] > (df['mean'] + 3 * df['std'])) | (df['valor'] < (df['mean'] - 3 * df['std']))

outliers = df[df['outlier']]
import IPython.display as display
display.display(outliers)

,id_da_compra,data,hora,cliente,produto,valor,quantidade,total,status,cidade,estado,pais,cep,frete,pagamento,vendedor,marca,mean,std,outlier
64,56909,2018-08-22,22:48:35,MARCO RIBEIRO AMORIM,arroz,586.88,6,26.88,aguardando pagamento,Santana do Garambéu,MG,Brasil,36196-000,8.5,BOLETO,FELIPE HENRIQUE COSTA BARNABE MARAZO,Urbano,17.072630,87.147313,True
99,21247,2022-01-20,16:06:33,MARCO SANTOS DA SILVA,macarrao,454.57,13,55.11,aguardando pagamento,Lagoa Dourada,MG,Brasil,36345-000,10.0,CARTAO DE CREDITO,MICAEL MALAQUIAS DE SOUZA OLIVEIRA,Adria,9.711984,51.597897,True
122,59643,2020-08-02,07:21:37,MATEUS VITOR FAÇANHA NEVES,sabonete,227.94,9,15.66,aguardando pagamento,Niterói,RJ,Brasil,24000-000,20.0,PIX,PAULO SOUZA RONCETE,Dove,7.142658,37.896064,True
134,31322,2021-02-17,11:01:36,GABRIEL MATOS LIMA DA CUNHA,manteiga,923.55,7,49.35,em separacao,Ritápolis,MG,Brasil,36335-000,9.0,BOLETO,HENRICO VICTOR ALVES,Itambé,27.220375,140.838513,True
179,11658,2020-08-04,12:01:19,FELIPE SILVINO DA SILVA,amaciante,1002.15,8,61.20,pagamento confirmado,Ouro Preto,MG,Brasil,35400-000,12.5,PIX,MICAEL MALAQUIAS DE SOUZA OLIVEIRA,Confort,20.018984,103.032062,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
368655,18666,2020-07-16,05:06:36,HENRIQUE HENRIQUE CAÇADOR,sabonete,153.27,5,23.85,pagamento confirmado,São Bernardo do Campo,SP,Brasil,09700-000,18.0,PIX,VICTOR GONÇALVES DONADONI,Lux,6.776570,36.052814,True
368661,21327,2022-07-15,07:20:22,MATHEUS LACERDA PORTUGAL DA SILVA,detergente,480.77,15,55.05,em transporte,Santos Dumont,MG,Brasil,36240-000,11.0,TRANSFERENCIA BANCARIA,MICAEL MALAQUIAS DE SOUZA OLIVEIRA,Ypê,10.977885,56.315542,True
368729,47461,2019-06-01,07:53:51,SAMUEL MUZY FERNANDES,suco de laranja,989.05,1,7.55,entregue,Tiradentes,MG,Brasil,36325-000,10.0,CARTAO DE CREDITO,HENRICO VICTOR ALVES,Prats,22.421782,114.751404,True
368733,40352,2019-04-22,23:12:31,VICTOR RIBAS DE MOURA,queijo mussarela,2022.64,11,180.34,pagamento confirmado,Congonhas,MG,Brasil,36415-000,10.5,PIX,VICTOR GONÇALVES DONADONI,Porto Alegre,52.714207,270.196211,True


O código identifica preços muito fora do padrão para cada produto e marca, marcando como outlier quem está fora da média ± 3 desvios. Depois, ele mostra só esses casos.

In [185]:
df['total_calculado'] = df['valor'] * df['quantidade']
df['erro_total'] = df['total'] != df['total_calculado']

print(df[df['erro_total']])

        id_da_compra       data      hora                         cliente  \
0              13679 2021-03-20  23:35:51               LUCAS ARAUJO KUHN   
2              47484 2021-06-09  15:30:28       FELIPE AUGUSTO NERY SILVA   
3              20809 2022-06-04  08:41:23           LEVI RIBEIRO AMORIM     
5              38623 2018-02-19  17:32:01     LUCAS ANTÔNIO DE SOUZA NETO   
6              46627 2018-04-21  22:08:30  GUILHERME IVAIR DE ASSIS SETTA   
...              ...        ...       ...                             ...   
368743         31219 2022-07-30  12:46:40       HIAN IVAIR DE ASSIS SETTA   
368744         11220 2019-08-14  19:48:17         LETICIA VIEIRA DEMARQUE   
368745         43399 2018-11-02  23:00:58        PEDRO GONÇALVES DONADONI   
368746         18398 2022-08-13  12:46:43            PEDRO MURATORI DUTRA   
368748         16921 2020-04-25  06:59:09        SAMUEL JOSÉ SILVA MATIAS   

                 produto   valor  quantidade   total                status 


Esse código verifica se o valor total está correto, comparando valor * quantidade com a coluna total. Se estiver diferente, marca como erro e mostra os casos com esse problema.

In [186]:

media_desvio = df.groupby(['produto', 'marca'])['valor'].agg(['mean', 'std']).reset_index()

media_desvio.rename(columns={'mean': 'media', 'std': 'desvio'}, inplace=True)

df = df.merge(media_desvio, on=['produto', 'marca'], how='left')

df['outlier'] = (df['valor'] > (df['media'] + 3 * df['desvio'])) | (df['valor'] < (df['media'] - 3 * df['desvio']))


df.loc[df['outlier'], 'valor'] = df.loc[df['outlier'], 'media']

df = df.drop(columns=['media', 'desvio', 'outlier'])

print(df)

        id_da_compra       data      hora                            cliente  \
0              13679 2021-03-20  23:35:51                  LUCAS ARAUJO KUHN   
1              28070 2020-10-30  09:00:53               MICAEL SOUZA RONCETE   
2              47484 2021-06-09  15:30:28          FELIPE AUGUSTO NERY SILVA   
3              20809 2022-06-04  08:41:23              LEVI RIBEIRO AMORIM     
4              47123 2019-05-04  13:38:45        GABRIEL MATOS LIMA DA CUNHA   
...              ...        ...       ...                                ...   
368745         43399 2018-11-02  23:00:58           PEDRO GONÇALVES DONADONI   
368746         18398 2022-08-13  12:46:43               PEDRO MURATORI DUTRA   
368747         12169 2021-02-23  19:09:58         TIAGO IVAIR DE ASSIS SETTA   
368748         16921 2020-04-25  06:59:09           SAMUEL JOSÉ SILVA MATIAS   
368749         11240 2018-03-22  22:47:21  LUCAS MALAQUIAS DE SOUZA OLIVEIRA   

                 produto      valor  qu

Esse código detecta valores de preço (valor) fora do padrão por produto e marca (usando média ± 3 desvios) e corrige esses valores anômalos substituindo pela média correta. Depois, remove as colunas auxiliares usadas na verificação e exibe o DataFrame limpo.

In [187]:
df = df.drop(columns=['std','mean'])

deletando as colunas que foram criadas acima

In [188]:
print(df.isnull().sum())

id_da_compra          0
data                  0
hora                  0
cliente               0
produto               0
valor                 0
quantidade            0
total              3684
status                0
cidade                0
estado                0
pais                  0
cep                   0
frete                 0
pagamento             0
vendedor              0
marca                 0
total_calculado       0
erro_total            0
dtype: int64


verificando quais colunas tem nulos ainda

In [189]:
df.sort_values(by='cep')

,id_da_compra,data,hora,cliente,produto,valor,quantidade,total,status,cidade,estado,pais,cep,frete,pagamento,vendedor,marca,total_calculado,erro_total
50908,39097,2018-04-29,21:38:48,PAULO SANTOS DA SILVA,sabonete,1.95,6,31.70,aguardando pagamento,São Paulo,SP,Brasil,01000-000,20.0,BOLETO,HENRICO VICTOR ALVES,Lux,11.70,True
313643,12778,2021-02-27,17:04:07,MATHEUS ANTÔNIO DE DECCO OLIVEIRA,biscoito recheado,3.35,13,63.55,aguardando pagamento,São Paulo,SP,Brasil,01000-000,20.0,TRANSFERENCIA BANCARIA,HENRICO MATOS LIMA DA CUNHA,Bauducco,43.55,True
153730,12242,2020-01-01,12:36:29,HENRICO MATHEUS DA SILVA,desinfetante,4.78,2,9.56,pagamento confirmado,São Paulo,SP,Brasil,01000-000,20.0,CARTAO CREDITO A VISTA,FELIPE HENRIQUE COSTA BARNABE MARAZO,Pinho Sol,9.56,False
153834,33051,2019-01-09,09:20:20,MATEUS ARAUJO KUHN,detergente,1.75,12,41.00,em separacao,São Paulo,SP,Brasil,01000-000,20.0,PIX,HENRICO VICTOR ALVES,Minuano,21.00,True
153900,58726,2021-10-03,04:02:17,JOÃO MATHEUS DA SILVA,acucar,3.04,5,35.20,aguardando pagamento,São Paulo,SP,Brasil,01000-000,20.0,TRANSFERENCIA BANCARIA,FELIPE HENRIQUE COSTA BARNABE MARAZO,Caravelas,15.20,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
301368,18619,2021-10-06,02:00:37,SAMUEL HUGO CAZULA PEREIRA,molho de tomate,3.17,13,41.21,aguardando pagamento,Santo André,São Paulo,Brasil,X9XXXXXX,18.0,TRANSFERENCIA BANCARIA,VICTOR GONÇALVES DONADONI,Predilecta,41.21,False
263502,2474,2020-03-14,18:56:55,BRENO WILSTONY DE SOUZA MENDES,agua mineral,2.20,14,30.80,em transporte,Santo André,SP,Brasil,X9XXXXXX,18.0,CARTAO DE CREDITO,GABRIEL QUEIROZ DE AGUIAR,São Lourenço,30.80,True
141189,22158,2022-03-10,14:38:57,SAMUEL SILVA RAMOS,manteiga,8.26,14,133.64,entregue,Santo André,SP,Brasil,X9XXXXXX,18.0,BOLETO,CARLOS QUEIROZ DE AGUIAR,Aviação,115.64,True
342259,34400,2020-03-14,19:43:05,THIAGO RIBEIRO AMORIM,acucar,2.90,2,5.80,pagamento confirmado,Santo André,São Paulo,Brasil,X9XXXXXX,18.0,CARTAO DE CREDITO,MICAEL MALAQUIAS DE SOUZA OLIVEIRA,Itamarati,5.80,False


In [190]:
ceps = df['cep'].unique()

ceps_lista = ceps.tolist()

for cep in ceps_lista:
    print(cep)

24000-000
35420-000
28900-000
28000-000
36400-000
36340-000
01000-000
36600-000
36300-000
24400-000
09700-000
25000-000
27900-000
36570-000
36000-000
07000-000
36120-000
13000-000
36212-000
36280-000
36200-000
24800-000
36880-000
25600-000
36420-000
36320-000
36240-000
36770-000
25500-000
36500-000
36710-000
36123-000
25900-000
36108-000
361X8XXX
366XXXXX
36196-000
36780-000
36680-000
30000-000
36510-000
36415-000
37370-000
36330-000
36700-000
26100-000
36335-000
36345-000
20000-000
26000-000
36350-000
12200-000
27200-000
36140-000
09000-000
3677XXXX
36325XXX
35400-000
36345XXX
36325-000
23900-000
3688XXXX
36123XXX
3668XXXX
367XXXXX
244XXXXX
2XXXXXXX
X9XXXXXX
25XXXXXX
13XXXXXX
36335XXX
248XXXXX
3614XXXX
36415XXX
3678XXXX
256XXXXX
363XXXXX
365XXXXX
289XXXXX
36196XXX
259XXXXX
239XXXXX
3642XXXX
X7XXXXXX
272XXXXX
X1XXXXXX
3651XXXX
3628XXXX
354XXXXX
3633XXXX
3612XXXX
3737XXXX
3671XXXX
3634XXXX
122XXXXX
3542XXXX
3635XXXX
28XXXXXX
24XXXXXX
3624XXXX
36XXXXXX
261XXXXX
364XXXXX
362XXXXX
3XXXXXXX

exibindo cep em lista para melhor exibicao e ver quais tem dados errados

In [191]:
def validar_cep(cep):
    return bool(re.fullmatch(r'\d{5}-\d{3}', cep))

df['cep_invalido'] = ~df['cep'].apply(validar_cep)

ceps_por_cidade = df[~df['cep_invalido']].groupby('cidade')['cep'].agg(lambda x: x.mode()[0])

df.loc[df['cep_invalido'], 'cep'] = df.loc[df['cep_invalido'], 'cidade'].map(ceps_por_cidade)

print(df)

        id_da_compra       data      hora                            cliente  \
0              13679 2021-03-20  23:35:51                  LUCAS ARAUJO KUHN   
1              28070 2020-10-30  09:00:53               MICAEL SOUZA RONCETE   
2              47484 2021-06-09  15:30:28          FELIPE AUGUSTO NERY SILVA   
3              20809 2022-06-04  08:41:23              LEVI RIBEIRO AMORIM     
4              47123 2019-05-04  13:38:45        GABRIEL MATOS LIMA DA CUNHA   
...              ...        ...       ...                                ...   
368745         43399 2018-11-02  23:00:58           PEDRO GONÇALVES DONADONI   
368746         18398 2022-08-13  12:46:43               PEDRO MURATORI DUTRA   
368747         12169 2021-02-23  19:09:58         TIAGO IVAIR DE ASSIS SETTA   
368748         16921 2020-04-25  06:59:09           SAMUEL JOSÉ SILVA MATIAS   
368749         11240 2018-03-22  22:47:21  LUCAS MALAQUIAS DE SOUZA OLIVEIRA   

                 produto      valor  qu

validando ceps com os numeros o hifen e os numeros finais e criando uma coluna de cep invalidos 

In [192]:
df.to_csv('dados_corrigidos.csv', index=False, encoding='utf-8')

In [193]:
df.sort_values(by='cep')

,id_da_compra,data,hora,cliente,produto,valor,quantidade,total,status,cidade,estado,pais,cep,frete,pagamento,vendedor,marca,total_calculado,erro_total,cep_invalido
104075,38564,2020-04-29,11:17:40,MATHEUS CASTILHO CARDOSO,desinfetante,3.38,6,40.28,em separacao,São Paulo,SP,Brasil,01000-000,20.0,PIX,LUCAS VITOR FAÇANHA NEVES,Pinho Sol,20.28,True,False
300500,10081,2022-11-25,23:54:03,YURI ANTÔNIO DE DECCO OLIVEIRA,sabonete,2.49,10,44.90,aguardando pagamento,São Paulo,SP,Brasil,01000-000,20.0,PIX,PAULO SOUZA RONCETE,Nivea,24.90,True,False
41081,54250,2019-03-25,14:14:25,FELIPE DA SILVA BIANQUINI,suco de laranja,5.11,9,65.99,pagamento confirmado,São Paulo,SP,Brasil,01000-000,20.0,CARTAO DE CREDITO,LUCAS VITOR FAÇANHA NEVES,Dafruta,45.99,True,False
116154,34628,2019-05-04,16:29:40,RÔMULO AUGUSTO NERY SILVA,sabonete,2.85,3,28.55,pagamento confirmado,São Paulo,SP,Brasil,01000-000,20.0,BOLETO,HENRICO VICTOR ALVES,Dove,8.55,True,False
335270,50404,2021-05-15,17:27:01,GABRIEL BRENO PRADO SILVA,cafe,8.10,4,52.40,aguardando pagamento,São Paulo,SP,Brasil,01000-000,20.0,BOLETO,MICAEL MALAQUIAS DE SOUZA OLIVEIRA,Melitta,32.40,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98898,24384,2019-11-09,17:12:26,GUILHERME SANTOS DA SILVA,molho de tomate,4.38,7,NaN,pagamento confirmado,São Vicente de Minas,MG,Brasil,37370-000,12.0,TRANSFERENCIA BANCARIA,GABRIEL QUEIROZ DE AGUIAR,Fugini,30.66,True,False
17049,44322,2022-03-16,16:00:06,GABRIEL DA CUNHA TEIXEIRA,vinho,26.35,1,26.35,aguardando pagamento,São Vicente de Minas,MG,Brasil,37370-000,12.0,CARTAO DE CREDITO,FELIPE HENRIQUE COSTA BARNABE MARAZO,Concha y Toro,26.35,False,False
359933,2622,2019-01-06,07:42:06,PEDRO FELIPE DE SOUZA,cerveja,2.69,16,43.04,em transporte,São Vicente de Minas,MG,Brasil,37370-000,12.0,CARTAO DE CREDITO,HENRICO MATOS LIMA DA CUNHA,Skol,43.04,False,False
305585,22823,2021-01-30,13:17:02,VICTOR DA CUNHA TEIXEIRA,cafe,9.85,1,9.85,entregue,São Vicente de Minas,MG,Brasil,37370-000,12.0,CARTAO DE CREDITO,LUCAS VITOR FAÇANHA NEVES,3 Corações,9.85,False,False


In [194]:

from IPython.display import display

display(df.head(10)) 
display(df.isnull().sum())

,id_da_compra,data,hora,cliente,produto,valor,quantidade,total,status,cidade,estado,pais,cep,frete,pagamento,vendedor,marca,total_calculado,erro_total,cep_invalido
0,13679,2021-03-20,23:35:51,LUCAS ARAUJO KUHN,queijo mussarela,16.87,13,239.31,pagamento confirmado,Niterói,RJ,Brasil,24000-000,20.0,CARTAO DE CREDITO,SAMUEL HENRIQUE CAÇADOR,Porto Alegre,219.31,True,False
1,28070,2020-10-30,09:00:53,MICAEL SOUZA RONCETE,molho de tomate,3.25,3,9.75,pagamento confirmado,Mariana,MG,Brasil,35420-000,12.0,PIX,MICAEL MALAQUIAS DE SOUZA OLIVEIRA,Fugini,9.75,False,False
2,47484,2021-06-09,15:30:28,FELIPE AUGUSTO NERY SILVA,agua mineral,1.63,9,36.67,pagamento confirmado,Cabo Frio,RJ,Brasil,28900-000,22.0,TRANSFERENCIA BANCARIA,HENRICO MATOS LIMA DA CUNHA,Minalba,14.67,True,False
3,20809,2022-06-04,08:41:23,LEVI RIBEIRO AMORIM,carvao,8.74,4,54.96,em separacao,Campos dos Goytacazes,RJ,Brasil,28000-000,20.0,PIX,GABRIEL QUEIROZ DE AGUIAR,Marca-Brás,34.96,True,False
4,47123,2019-05-04,13:38:45,GABRIEL MATOS LIMA DA CUNHA,cafe,9.48,2,18.96,em separacao,Conselheiro Lafaiete,MG,Brasil,36400-000,10.5,CARTAO DE CREDITO,HENRICO MATOS LIMA DA CUNHA,3 Corações,18.96,False,False
5,38623,2018-02-19,17:32:01,LUCAS ANTÔNIO DE SOUZA NETO,cafe,10.16,1,20.16,entregue com sucesso,Resende Costa,MG,Brasil,36340-000,10.0,TRANSFERENCIA BANCARIA,VICTOR GONÇALVES DONADONI,3 Corações,10.16,True,False
6,46627,2018-04-21,22:08:30,GUILHERME IVAIR DE ASSIS SETTA,feijao,7.77,3,43.31,em separacao,São Paulo,SP,Brasil,01000-000,20.0,PIX,SAMUEL HENRIQUE CAÇADOR,Caldo Bom,23.31,True,False
7,53579,2020-09-25,12:04:15,GABRIEL MUZY FERNANDES,queijo mussarela,14.93,14,218.02,em separacao,Bicas,MG,Brasil,36600-000,9.0,BOLETO,MICAEL MALAQUIAS DE SOUZA OLIVEIRA,Porto Alegre,209.02,True,False
8,34924,2018-06-20,17:10:42,LEVI MATOS LIMA DA CUNHA,condicionador,9.51,5,69.55,pagamento confirmado,Cabo Frio,RJ,Brasil,28900-000,22.0,PIX,FELIPE HENRIQUE COSTA BARNABE MARAZO,Pantene,47.55,True,False
9,10575,2018-10-01,18:36:45,HENRICO DA CUNHA TEIXEIRA,sabonete,1.25,2,13.00,em separacao,São João del-Rei,MG,Brasil,36300-000,10.5,BOLETO,GABRIEL QUEIROZ DE AGUIAR,Dove,2.50,True,False


id_da_compra          0
data                  0
hora                  0
cliente               0
produto               0
valor                 0
quantidade            0
total              3684
status                0
cidade                0
estado                0
pais                  0
cep                   0
frete                 0
pagamento             0
vendedor              0
marca                 0
total_calculado       0
erro_total            0
cep_invalido          0
dtype: int64

In [195]:
df.sort_values(by='frete')

,id_da_compra,data,hora,cliente,produto,valor,quantidade,total,status,cidade,estado,pais,cep,frete,pagamento,vendedor,marca,total_calculado,erro_total,cep_invalido
155141,20746,2021-09-13,17:47:06,BLENDOW AUGUSTO NERY SILVA,refrigerante,6.01,2,20.02,em separacao,Prados,MG,Brasil,36320-000,8.0,PIX,GABRIEL QUEIROZ DE AGUIAR,Coca-Cola,12.02,True,False
75464,7522,2018-06-30,18:34:51,LUCAS PAULO ALVES ALVIM,queijo mussarela,16.67,9,158.03,em separacao,Prados,MG,Brasil,36320-000,8.0,PIX,LUCAS VITOR FAÇANHA NEVES,Tirolez,150.03,True,False
168981,56965,2021-03-16,13:51:27,HIAN RIBAS DE MOURA,oleo de soja,4.07,1,4.07,aguardando pagamento,Prados,MG,Brasil,36320-000,8.0,CARTAO DE CREDITO,MICAEL MALAQUIAS DE SOUZA OLIVEIRA,Coamo,4.07,False,False
75456,52970,2020-03-15,07:43:34,HENRICO MATHEUS DA SILVA,desinfetante,3.67,15,55.05,pagamento confirmado,Prados,MG,Brasil,36320-000,8.0,CARTAO DE CREDITO,PAULO SOUZA RONCETE,Veja,55.05,False,False
318956,48518,2019-04-04,09:37:11,JOÃO ANTÔNIO DE SOUZA NETO,refrigerante,6.57,2,13.14,em separacao,Prados,MG,Brasil,36320-000,8.0,CARTAO DE CREDITO,HENRICO VICTOR ALVES,Coca-Cola,13.14,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87383,46427,2020-02-12,16:45:44,LUCAS RIBAS DE MOURA,arroz,5.04,5,38.20,em separacao,Juiz de Fora,MG,Brasil,36000-000,91650.0,PIX,VICTOR GONÇALVES DONADONI,Camil,25.20,True,False
51655,43285,2019-10-12,17:49:14,RÔMULO RIBAS DE MOURA,pao de forma,6.61,2,26.22,pagamento confirmado,Juiz de Fora,MG,Brasil,36000-000,91650.0,BOLETO,GABRIEL QUEIROZ DE AGUIAR,Nutrella,13.22,True,False
5214,24436,2021-03-25,07:52:49,LETICIA CARLOS DE SOUZA,sabonete,2.21,4,25.84,pagamento confirmado,São José dos Campos,SP,Brasil,12200-000,119850.0,CARTAO DE CREDITO,HENRICO MATOS LIMA DA CUNHA,Nivea,8.84,True,False
167239,37503,2020-04-17,20:42:48,BLENDOW ARAUJO KUHN,biscoito recheado,3.06,13,58.78,pagamento confirmado,Volta Redonda,RJ,Brasil,27200-000,133950.0,PIX,MICAEL MALAQUIAS DE SOUZA OLIVEIRA,Bauducco,39.78,True,False


In [196]:
frete_para_cidades = df.groupby('cidade')['frete'].unique()
print(frete_para_cidades)

cidade
Angra dos Reis           [23.0, 230.0, 16215.0, 162150.0]
Astolfo Dutra                       [10.0, 7050.0, 100.0]
Barbacena                           [12.0, 8460.0, 120.0]
Barroso                               [8.5, 5992.5, 85.0]
Belford Roxo                       [19.0, 13395.0, 190.0]
Belo Horizonte             [10.0, 100.0, 7050.0, 70500.0]
Bicas                                 [9.0, 6345.0, 90.0]
Cabo Frio                          [22.0, 15510.0, 220.0]
Campinas                           [17.0, 11985.0, 170.0]
Campos dos Goytacazes              [20.0, 200.0, 14100.0]
Carandaí                              [9.0, 6345.0, 90.0]
Cataguases                 [10.5, 7402.5, 105.0, 74025.0]
Congonhas                           [10.5, 105.0, 7402.5]
Conselheiro Lafaiete                [10.5, 7402.5, 105.0]
Coronel Xavier Chaves               [7755.0, 11.0, 110.0]
Duque de Caxias                    [19.0, 190.0, 13395.0]
Ewbank da Câmara                    [11.0, 7755.0, 110.0]
Guarulh

Verificando os valores de fretes por cada cidade

In [197]:
valor_comum = df.groupby('cidade')['frete'].agg(lambda x: x.mode().iloc[0])
print(valor_comum)


cidade
Angra dos Reis           23.0
Astolfo Dutra            10.0
Barbacena                12.0
Barroso                   8.5
Belford Roxo             19.0
Belo Horizonte           10.0
Bicas                     9.0
Cabo Frio                22.0
Campinas                 17.0
Campos dos Goytacazes    20.0
Carandaí                  9.0
Cataguases               10.5
Congonhas                10.5
Conselheiro Lafaiete     10.5
Coronel Xavier Chaves    11.0
Duque de Caxias          19.0
Ewbank da Câmara         11.0
Guarulhos                18.0
Itaboraí                 18.5
Juiz de Fora             13.0
Lagoa Dourada            10.0
Leopoldina                9.5
Lima Duarte              10.0
Macaé                    20.0
Magé                     18.0
Mariana                  12.0
Matias Barbosa           10.0
Muriaé                   10.0
Niterói                  20.0
Nova Iguaçu              18.0
Ouro Branco              10.0
Ouro Preto               12.5
Palma                     9.0
Pet

Esse código pega o valor do frete mais comum (moda) para cada cidade. Ele agrupa por cidade e, dentro de cada grupo, identifica qual valor aparece com mais frequência. Isso é útil, por exemplo, pra preencher dados ausentes com o valor mais representativo da cidade.

In [198]:
df['frete'] = df['cidade'].map(valor_comum)
print(df)

        id_da_compra       data      hora                            cliente  \
0              13679 2021-03-20  23:35:51                  LUCAS ARAUJO KUHN   
1              28070 2020-10-30  09:00:53               MICAEL SOUZA RONCETE   
2              47484 2021-06-09  15:30:28          FELIPE AUGUSTO NERY SILVA   
3              20809 2022-06-04  08:41:23              LEVI RIBEIRO AMORIM     
4              47123 2019-05-04  13:38:45        GABRIEL MATOS LIMA DA CUNHA   
...              ...        ...       ...                                ...   
368745         43399 2018-11-02  23:00:58           PEDRO GONÇALVES DONADONI   
368746         18398 2022-08-13  12:46:43               PEDRO MURATORI DUTRA   
368747         12169 2021-02-23  19:09:58         TIAGO IVAIR DE ASSIS SETTA   
368748         16921 2020-04-25  06:59:09           SAMUEL JOSÉ SILVA MATIAS   
368749         11240 2018-03-22  22:47:21  LUCAS MALAQUIAS DE SOUZA OLIVEIRA   

                 produto      valor  qu

In [199]:
df.isnull().sum()

id_da_compra          0
data                  0
hora                  0
cliente               0
produto               0
valor                 0
quantidade            0
total              3684
status                0
cidade                0
estado                0
pais                  0
cep                   0
frete                 0
pagamento             0
vendedor              0
marca                 0
total_calculado       0
erro_total            0
cep_invalido          0
dtype: int64

In [200]:
display(df.head(10))

,id_da_compra,data,hora,cliente,produto,valor,quantidade,total,status,cidade,estado,pais,cep,frete,pagamento,vendedor,marca,total_calculado,erro_total,cep_invalido
0,13679,2021-03-20,23:35:51,LUCAS ARAUJO KUHN,queijo mussarela,16.87,13,239.31,pagamento confirmado,Niterói,RJ,Brasil,24000-000,20.0,CARTAO DE CREDITO,SAMUEL HENRIQUE CAÇADOR,Porto Alegre,219.31,True,False
1,28070,2020-10-30,09:00:53,MICAEL SOUZA RONCETE,molho de tomate,3.25,3,9.75,pagamento confirmado,Mariana,MG,Brasil,35420-000,12.0,PIX,MICAEL MALAQUIAS DE SOUZA OLIVEIRA,Fugini,9.75,False,False
2,47484,2021-06-09,15:30:28,FELIPE AUGUSTO NERY SILVA,agua mineral,1.63,9,36.67,pagamento confirmado,Cabo Frio,RJ,Brasil,28900-000,22.0,TRANSFERENCIA BANCARIA,HENRICO MATOS LIMA DA CUNHA,Minalba,14.67,True,False
3,20809,2022-06-04,08:41:23,LEVI RIBEIRO AMORIM,carvao,8.74,4,54.96,em separacao,Campos dos Goytacazes,RJ,Brasil,28000-000,20.0,PIX,GABRIEL QUEIROZ DE AGUIAR,Marca-Brás,34.96,True,False
4,47123,2019-05-04,13:38:45,GABRIEL MATOS LIMA DA CUNHA,cafe,9.48,2,18.96,em separacao,Conselheiro Lafaiete,MG,Brasil,36400-000,10.5,CARTAO DE CREDITO,HENRICO MATOS LIMA DA CUNHA,3 Corações,18.96,False,False
5,38623,2018-02-19,17:32:01,LUCAS ANTÔNIO DE SOUZA NETO,cafe,10.16,1,20.16,entregue com sucesso,Resende Costa,MG,Brasil,36340-000,10.0,TRANSFERENCIA BANCARIA,VICTOR GONÇALVES DONADONI,3 Corações,10.16,True,False
6,46627,2018-04-21,22:08:30,GUILHERME IVAIR DE ASSIS SETTA,feijao,7.77,3,43.31,em separacao,São Paulo,SP,Brasil,01000-000,20.0,PIX,SAMUEL HENRIQUE CAÇADOR,Caldo Bom,23.31,True,False
7,53579,2020-09-25,12:04:15,GABRIEL MUZY FERNANDES,queijo mussarela,14.93,14,218.02,em separacao,Bicas,MG,Brasil,36600-000,9.0,BOLETO,MICAEL MALAQUIAS DE SOUZA OLIVEIRA,Porto Alegre,209.02,True,False
8,34924,2018-06-20,17:10:42,LEVI MATOS LIMA DA CUNHA,condicionador,9.51,5,69.55,pagamento confirmado,Cabo Frio,RJ,Brasil,28900-000,22.0,PIX,FELIPE HENRIQUE COSTA BARNABE MARAZO,Pantene,47.55,True,False
9,10575,2018-10-01,18:36:45,HENRICO DA CUNHA TEIXEIRA,sabonete,1.25,2,13.00,em separacao,São João del-Rei,MG,Brasil,36300-000,10.5,BOLETO,GABRIEL QUEIROZ DE AGUIAR,Dove,2.50,True,False


In [201]:
df = df.drop(columns=['cep_invalido','total_calculado','erro_total'])


deletando colunas que nao serao usadas mais 

In [202]:
display(df.head(10))

,id_da_compra,data,hora,cliente,produto,valor,quantidade,total,status,cidade,estado,pais,cep,frete,pagamento,vendedor,marca
0,13679,2021-03-20,23:35:51,LUCAS ARAUJO KUHN,queijo mussarela,16.87,13,239.31,pagamento confirmado,Niterói,RJ,Brasil,24000-000,20.0,CARTAO DE CREDITO,SAMUEL HENRIQUE CAÇADOR,Porto Alegre
1,28070,2020-10-30,09:00:53,MICAEL SOUZA RONCETE,molho de tomate,3.25,3,9.75,pagamento confirmado,Mariana,MG,Brasil,35420-000,12.0,PIX,MICAEL MALAQUIAS DE SOUZA OLIVEIRA,Fugini
2,47484,2021-06-09,15:30:28,FELIPE AUGUSTO NERY SILVA,agua mineral,1.63,9,36.67,pagamento confirmado,Cabo Frio,RJ,Brasil,28900-000,22.0,TRANSFERENCIA BANCARIA,HENRICO MATOS LIMA DA CUNHA,Minalba
3,20809,2022-06-04,08:41:23,LEVI RIBEIRO AMORIM,carvao,8.74,4,54.96,em separacao,Campos dos Goytacazes,RJ,Brasil,28000-000,20.0,PIX,GABRIEL QUEIROZ DE AGUIAR,Marca-Brás
4,47123,2019-05-04,13:38:45,GABRIEL MATOS LIMA DA CUNHA,cafe,9.48,2,18.96,em separacao,Conselheiro Lafaiete,MG,Brasil,36400-000,10.5,CARTAO DE CREDITO,HENRICO MATOS LIMA DA CUNHA,3 Corações
5,38623,2018-02-19,17:32:01,LUCAS ANTÔNIO DE SOUZA NETO,cafe,10.16,1,20.16,entregue com sucesso,Resende Costa,MG,Brasil,36340-000,10.0,TRANSFERENCIA BANCARIA,VICTOR GONÇALVES DONADONI,3 Corações
6,46627,2018-04-21,22:08:30,GUILHERME IVAIR DE ASSIS SETTA,feijao,7.77,3,43.31,em separacao,São Paulo,SP,Brasil,01000-000,20.0,PIX,SAMUEL HENRIQUE CAÇADOR,Caldo Bom
7,53579,2020-09-25,12:04:15,GABRIEL MUZY FERNANDES,queijo mussarela,14.93,14,218.02,em separacao,Bicas,MG,Brasil,36600-000,9.0,BOLETO,MICAEL MALAQUIAS DE SOUZA OLIVEIRA,Porto Alegre
8,34924,2018-06-20,17:10:42,LEVI MATOS LIMA DA CUNHA,condicionador,9.51,5,69.55,pagamento confirmado,Cabo Frio,RJ,Brasil,28900-000,22.0,PIX,FELIPE HENRIQUE COSTA BARNABE MARAZO,Pantene
9,10575,2018-10-01,18:36:45,HENRICO DA CUNHA TEIXEIRA,sabonete,1.25,2,13.00,em separacao,São João del-Rei,MG,Brasil,36300-000,10.5,BOLETO,GABRIEL QUEIROZ DE AGUIAR,Dove


In [203]:
df['total_sem_frete'] = df['valor'] * df['quantidade']
df['total'] = (df['valor'] * df['quantidade']) + df['frete']

Esse código primeiro calcula o total sem o frete (valor * quantidade) e depois atualiza a coluna total com o valor incluindo o frete. Assim, garante que o total final de cada compra esteja correto.

In [204]:
dados_duplicados = df[df.duplicated(subset=['id_da_compra'])]
print(dados_duplicados)

        id_da_compra       data      hora                            cliente  \
82             56548 2018-05-03  14:47:29    MARCO LACERDA PORTUGAL DA SILVA   
511            28778 2022-04-15  10:31:38               YURI VIEIRA DEMARQUE   
563            31676 2022-07-02  16:59:37         HENRIQUE CUNHA DE OLIVEIRA   
720            51030 2018-11-01  21:04:23                BRENO CARDOSO DINIZ   
724            57167 2020-06-19  11:56:18         MATHEUS AUGUSTO NERY SILVA   
...              ...        ...       ...                                ...   
368745         43399 2018-11-02  23:00:58           PEDRO GONÇALVES DONADONI   
368746         18398 2022-08-13  12:46:43               PEDRO MURATORI DUTRA   
368747         12169 2021-02-23  19:09:58         TIAGO IVAIR DE ASSIS SETTA   
368748         16921 2020-04-25  06:59:09           SAMUEL JOSÉ SILVA MATIAS   
368749         11240 2018-03-22  22:47:21  LUCAS MALAQUIAS DE SOUZA OLIVEIRA   

             produto      valor  quanti

Esse código identifica e exibe as linhas duplicadas com base no id_da_compra — ou seja, compras que aparecem mais de uma vez na base. É útil pra verificar se há registros repetidos da mesma compra.

In [205]:
df_sorted = df.copy()
df_sorted['duplicado'] = df_sorted.duplicated(subset=['id_da_compra'], keep=False)  
df_sorted = df_sorted[df_sorted['duplicado']]  
df_sorted = df_sorted.sort_values(by='id_da_compra')
display(df_sorted.head(15))

,id_da_compra,data,hora,cliente,produto,valor,quantidade,total,status,cidade,estado,pais,cep,frete,pagamento,vendedor,marca,total_sem_frete,duplicado
364789,0,2019-07-07,11:05:44,PEDRO SILVA RAMOS,pasta de dente,2.98000,9,36.820000,aguardando pagamento,Ouro Branco,MG,Brasil,36420-000,10.0,CARTAO DE CREDITO,VICTOR GONÇALVES DONADONI,Colgate,26.820000,True
143430,0,2019-07-07,11:05:44,PEDRO SILVA RAMOS,sabonete,2.40000,15,46.000000,aguardando pagamento,Ouro Branco,MG,Brasil,36420-000,10.0,CARTAO DE CREDITO,VICTOR GONÇALVES DONADONI,Nivea,36.000000,True
119875,0,2019-07-07,11:05:44,PEDRO SILVA RAMOS,sabonete,6.77657,7,57.435988,aguardando pgto,Ouro Branco,MG,Brasil,36420-000,10.0,CARTAO DE CREDITO,VICTOR GONÇALVES DONADONI,Lux,47.435988,True
4853,0,2019-07-07,11:05:44,PEDRO SILVA RAMOS,macarrao,2.63000,14,46.820000,aguardando pagamento,Ouro Branco,MG,Brasil,36420-000,10.0,CARTAO DE CREDITO,VICTOR GONÇALVES DONADONI,Renata,36.820000,True
170362,1,2018-08-23,17:52:53,PEDRO DA SILVA BIANQUINI,sabonete,2.37000,9,40.330000,pagamento confirmado,Duque de Caxias,RJ,Brasil,25000-000,19.0,CARTAO DE CREDITO,HENRICO VICTOR ALVES,Dove,21.330000,True
158809,1,2018-08-23,17:52:53,PEDRO DA SILVA BIANQUINI,refrigerante,6.83000,5,53.150000,pagamento confirmado,Duque de Caxias,RJ,Brasil,25000-000,19.0,CARTAO DE CREDITO,HENRICO VICTOR ALVES,Fanta,34.150000,True
102490,1,2018-08-23,17:52:53,PEDRO DA SILVA BIANQUINI,detergente,3.45000,15,70.750000,pagamento confirmado,Duque de Caxias,RJ,Brasil,25000-000,19.0,CARTAO DE CREDITO,HENRICO VICTOR ALVES,Minuano,51.750000,True
219752,1,2018-08-23,17:52:53,PEDRO DA SILVA BIANQUINI,macarrao,3.23000,8,44.840000,pagamento confirmado,Duque de Caxias,RJ,Brasil,25000-000,19.0,CARTAO DE CREDITO,HENRICO VICTOR ALVES,Barilla,25.840000,True
218036,2,2022-11-06,07:03:58,LUCAS CUNHA DE OLIVEIRA,acucar,2.13000,2,13.260000,aguardando pagamento,Viçosa,MG,Brasil,36570-000,9.0,TRANSFERENCIA BANCARIA,MICAEL MALAQUIAS DE SOUZA OLIVEIRA,União,4.260000,True
1154,2,2022-11-06,07:03:58,LUCAS CUNHA DE OLIVEIRA,shampoo,14.52000,12,183.240000,aguardando pagamento,Viçosa,MG,Brasil,36570-000,9.0,TRANSFERENCIA BANCARIA,MICAEL MALAQUIAS DE SOUZA OLIVEIRA,Pantene,174.240000,True


Esse código copia o DataFrame, marca todas as linhas que têm id_da_compra repetido (incluindo todas as ocorrências), filtra só essas duplicadas e ordena pelo id_da_compra. Por fim, mostra as 15 primeiras. É uma forma organizada de visualizar todas as compras duplicadas agrupadas.

In [206]:
df.isnull().sum()

id_da_compra       0
data               0
hora               0
cliente            0
produto            0
valor              0
quantidade         0
total              0
status             0
cidade             0
estado             0
pais               0
cep                0
frete              0
pagamento          0
vendedor           0
marca              0
total_sem_frete    0
dtype: int64

In [207]:
df.to_csv('dados_corrigidos.csv', index=False)

In [208]:
df_null = df[df[['valor', 'total', 'total_sem_frete']].isnull().any(axis=1)]
df.loc[df['total_sem_frete'].isnull(), 'total_sem_frete'] = df['quantidade'] * df['valor']
print(df_null)

Empty DataFrame
Columns: [id_da_compra, data, hora, cliente, produto, valor, quantidade, total, status, cidade, estado, pais, cep, frete, pagamento, vendedor, marca, total_sem_frete]
Index: []


Esse código identifica linhas onde valor, total ou total_sem_frete estão nulos e salva em df_null. Em seguida, preenche os valores ausentes de total_sem_frete com valor * quantidade. Por fim, imprime os dados que tinham pelo menos um desses campos nulos. É útil pra corrigir e revisar registros incompletos.

In [209]:
df.loc[df['total'].isnull(), 'total'] = (df['quantidade'] * df['valor']) + df['frete']

Esse código preenche os valores nulos na coluna total, calculando o valor correto com base em quantidade * valor + frete. Assim, garante que o total da compra esteja completo mesmo se estiver faltando.

In [210]:
df['valor'] = df.groupby(['produto', 'marca'])['valor'].transform(lambda x: x.fillna(x.mean()))

Esse código preenche os valores nulos na coluna valor, usando a média de preço do mesmo produto e marca. Ou seja, se o valor estiver faltando, ele usa a média do grupo correspondente pra completar. É uma forma inteligente de manter consistência nos dados.

In [211]:
df.isnull().sum()

id_da_compra       0
data               0
hora               0
cliente            0
produto            0
valor              0
quantidade         0
total              0
status             0
cidade             0
estado             0
pais               0
cep                0
frete              0
pagamento          0
vendedor           0
marca              0
total_sem_frete    0
dtype: int64

In [212]:
df.to_csv('dados_corrigidos.csv', index=False)

In [213]:
duplicadas = df.duplicated()
print(duplicadas.sum())
df[duplicadas]

5813


,id_da_compra,data,hora,cliente,produto,valor,quantidade,total,status,cidade,estado,pais,cep,frete,pagamento,vendedor,marca,total_sem_frete
7115,22833,2019-11-04,23:54:47,HENRIQUE AUGUSTO NERY SILVA,macarrao,3.39,4,32.06,aguardando pagamento,Itaboraí,RJ,Brasil,24800-000,18.5,TRANSFERENCIA BANCARIA,GABRIEL QUEIROZ DE AGUIAR,Renata,13.56
9451,37102,2020-08-02,10:32:27,JOÃO SILVINO DA SILVA,shampoo,7.65,12,100.80,em separacao,Viçosa,MG,Brasil,36570-000,9.0,CARTAO DE CREDITO,GABRIEL QUEIROZ DE AGUIAR,Pantene,91.80
11787,43901,2019-07-20,13:10:10,KALLEBE RIBEIRO AMORIM,desinfetante,5.97,6,45.32,em separacao,São Tiago,MG,Brasil,36350-000,9.5,PIX,CARLOS QUEIROZ DE AGUIAR,Veja,35.82
13994,5337,2020-05-24,12:10:42,GABRIEL VIEIRA DEMARQUE,desinfetante,6.34,2,29.68,em transporte,São José dos Campos,SP,Brasil,12200-000,17.0,CARTAO DE CREDITO,HENRICO VICTOR ALVES,Veja,12.68
14387,2852,2018-08-26,17:38:01,MICAEL SEGURASSE GONÇALVES,vinho,32.69,11,381.59,pagamento confirmado,Rio de Janeiro,RJ,Brasil,20000-000,22.0,TRANSFERENCIA BANCARIA,HENRICO MATOS LIMA DA CUNHA,Concha y Toro,359.59
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
368474,47230,2019-06-22,16:18:20,MATHEUS ANTÔNIO DE DECCO OLIVEIRA,desinfetante,5.64,15,94.60,em separacao,Resende Costa,MG,Brasil,36340-000,10.0,CARTAO DE CREDITO,PAULO SOUZA RONCETE,Pinho Sol,84.60
368501,50402,2022-06-25,08:13:33,GABRIEL WERNECK ELIZEU,cerveja,2.54,8,30.82,em separacao,Conselheiro Lafaiete,MG,Brasil,36400-000,10.5,CARTAO DE CREDITO,HENRICO VICTOR ALVES,Brahma,20.32
368601,31673,2020-12-31,19:24:49,ARTHUR WERNECK ELIZEU,shampoo,9.19,13,127.97,pagamento confirmado,Tocantins,MG,Brasil,36510-000,8.5,CARTAO DE CREDITO,GABRIEL QUEIROZ DE AGUIAR,Pantene,119.47
368626,14688,2019-01-06,18:01:24,YURI HENRIQUE COSTA BARNABE MARAZO,pasta de dente,3.45,8,37.60,em separacao,Matias Barbosa,MG,Brasil,36120-000,10.0,CARTAO DE CREDITO,FELIPE HENRIQUE COSTA BARNABE MARAZO,Sorriso,27.60


In [214]:
itens_duplicados = df.duplicated(subset=['id_da_compra','produto', 'marca'])
print(itens_duplicados.sum())
df[itens_duplicados].sort_values(by='id_da_compra')

34131


,id_da_compra,data,hora,cliente,produto,valor,quantidade,total,status,cidade,estado,pais,cep,frete,pagamento,vendedor,marca,total_sem_frete
125688,3,2021-03-05,22:45:31,PEDRO CARLOS DE SOUZA,queijo mussarela,16.09,13,229.17,pagamento confirmado,São Gonçalo,RJ,Brasil,24400-000,20.0,PIX,FELIPE HENRIQUE COSTA BARNABE MARAZO,Tirolez,209.17
214317,4,2018-03-13,12:17:20,SAMUEL DA SILVA BIANQUINI,suco de laranja,5.11,2,20.72,entregue,São João del-Rei,MG,Brasil,36300-000,10.5,TRANSFERENCIA BANCARIA,MICAEL MALAQUIAS DE SOUZA OLIVEIRA,Dafruta,10.22
242063,4,2018-03-13,12:17:20,SAMUEL DA SILVA BIANQUINI,refrigerante,4.45,6,37.20,entregue,São João del-Rei,MG,Brasil,36300-000,10.5,TRANSFERENCIA BANCARIA,MICAEL MALAQUIAS DE SOUZA OLIVEIRA,Fanta,26.70
137464,4,2018-03-13,12:17:20,SAMUEL DA SILVA BIANQUINI,refrigerante,5.25,2,21.00,entregue,São João del-Rei,MG,Brasil,36300-000,10.5,TRANSFERENCIA BANCARIA,MICAEL MALAQUIAS DE SOUZA OLIVEIRA,Fanta,10.50
62571,8,2018-02-22,14:41:03,JOÃO QUEIROZ DE AGUIAR,manteiga,8.07,13,124.91,aguardando pagamento,São Paulo,SP,Brasil,01000-000,20.0,TRANSFERENCIA BANCARIA,GABRIEL QUEIROZ DE AGUIAR,Aviação,104.91
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
211664,59985,2020-12-25,16:58:02,GABRIEL MATOS LIMA DA CUNHA,sabonete,2.60,10,46.00,em transporte,Niterói,RJ,Brasil,24000-000,20.0,TRANSFERENCIA BANCARIA,HENRICO MATOS LIMA DA CUNHA,Lux,26.00
273235,59991,2021-10-08,10:15:04,PAULO HUGO CAZULA PEREIRA,cerveja,3.98,21,94.08,separando,Cataguases,MG,Brasil,36770-000,10.5,TRANSFERENCIA BANCARIA,LUCAS VITOR FAÇANHA NEVES,Skol,83.58
167523,59996,2020-05-08,14:54:25,GABRIEL GONÇALVES DONADONI,cafe,9.13,2,27.26,em transporte,Palma,MG,Brasil,36710-000,9.0,PIX,MICAEL MALAQUIAS DE SOUZA OLIVEIRA,Melitta,18.26
360230,59996,2020-05-08,14:54:25,GABRIEL GONÇALVES DONADONI,cafe,8.43,2,25.86,em transporte,Palma,MG,Brasil,36710-000,9.0,PIX,MICAEL MALAQUIAS DE SOUZA OLIVEIRA,Melitta,16.86


Esse código procura por itens repetidos dentro da mesma compra, ou seja, casos em que o mesmo produto e marca aparecem mais de uma vez no mesmo id_da_compra. Ele conta quantas duplicatas existem e mostra esses registros organizados pela compra, ajudando a identificar repetições que não deveriam estar ali.

In [215]:
df = df.groupby(
    ['id_da_compra','data','hora','cliente','produto','valor','quantidade','total','status','cidade','estado','pais','cep','frete','pagamento','vendedor','marca','total_sem_frete'], 
    as_index=False
).agg({'quantidade': 'sum','valor': 'first','total': 'first','total_sem_frete': 'first'})

Esse código agrupa os dados por todas as colunas que definem uma compra completa e, caso existam repetições, ele soma as quantidades e mantém o primeiro valor encontrado para as outras colunas. Isso serve para eliminar duplicatas de forma consolidada, garantindo que a quantidade final esteja correta sem perder informações importantes.

In [216]:
df.loc[df['total'].isnull(), 'total'] = (df['quantidade'] * df['valor']) + df['frete']
df.loc[df['total_sem_frete'].isnull(), 'total_sem_frete'] = df['quantidade'] * df['valor']

Esse código preenche valores nulos nas colunas total e total_sem_frete. Se total estiver faltando, ele calcula com base em quantidade * valor + frete. Se total_sem_frete estiver faltando, calcula apenas quantidade * valor. Assim, ele corrige e completa os totais faltantes das compras.

In [217]:
df.to_csv('dados_corrigidos.csv', index=False)

In [218]:
df['cliente'] = df['cliente'].str.strip()  


removendo espaços no inicio e no fim 

In [219]:
df['vendedor'] = df['vendedor'].str.strip()  

removendo espaços no inicio e no fim 

In [220]:
df['marca'] = df['marca'].str.strip()  

removendo espaços no inicio e no fim 

In [221]:
display(df.head(10))


,id_da_compra,data,hora,cliente,produto,status,cidade,estado,pais,cep,frete,pagamento,vendedor,marca,quantidade,valor,total,total_sem_frete
0,0,2019-07-07,11:05:44,PEDRO SILVA RAMOS,macarrao,aguardando pagamento,Ouro Branco,MG,Brasil,36420-000,10.0,CARTAO DE CREDITO,VICTOR GONÇALVES DONADONI,Renata,14,2.63000,46.820000,36.820000
1,0,2019-07-07,11:05:44,PEDRO SILVA RAMOS,pasta de dente,aguardando pagamento,Ouro Branco,MG,Brasil,36420-000,10.0,CARTAO DE CREDITO,VICTOR GONÇALVES DONADONI,Colgate,9,2.98000,36.820000,26.820000
2,0,2019-07-07,11:05:44,PEDRO SILVA RAMOS,sabonete,aguardando pagamento,Ouro Branco,MG,Brasil,36420-000,10.0,CARTAO DE CREDITO,VICTOR GONÇALVES DONADONI,Nivea,15,2.40000,46.000000,36.000000
3,0,2019-07-07,11:05:44,PEDRO SILVA RAMOS,sabonete,aguardando pgto,Ouro Branco,MG,Brasil,36420-000,10.0,CARTAO DE CREDITO,VICTOR GONÇALVES DONADONI,Lux,7,6.77657,57.435988,47.435988
4,1,2018-08-23,17:52:53,PEDRO DA SILVA BIANQUINI,sabonete,pagamento confirmado,Duque de Caxias,RJ,Brasil,25000-000,19.0,CARTAO DE CREDITO,HENRICO VICTOR ALVES,Dove,9,2.37000,40.330000,21.330000
5,1,2018-08-23,17:52:53,PEDRO DA SILVA BIANQUINI,detergente,pagamento confirmado,Duque de Caxias,RJ,Brasil,25000-000,19.0,CARTAO DE CREDITO,HENRICO VICTOR ALVES,Minuano,15,3.45000,70.750000,51.750000
6,1,2018-08-23,17:52:53,PEDRO DA SILVA BIANQUINI,macarrao,pagamento confirmado,Duque de Caxias,RJ,Brasil,25000-000,19.0,CARTAO DE CREDITO,HENRICO VICTOR ALVES,Barilla,8,3.23000,44.840000,25.840000
7,1,2018-08-23,17:52:53,PEDRO DA SILVA BIANQUINI,refrigerante,pagamento confirmado,Duque de Caxias,RJ,Brasil,25000-000,19.0,CARTAO DE CREDITO,HENRICO VICTOR ALVES,Fanta,5,6.83000,53.150000,34.150000
8,2,2022-11-06,07:03:58,LUCAS CUNHA DE OLIVEIRA,acucar,aguardando pagamento,Viçosa,MG,Brasil,36570-000,9.0,TRANSFERENCIA BANCARIA,MICAEL MALAQUIAS DE SOUZA OLIVEIRA,União,2,2.13000,13.260000,4.260000
9,2,2022-11-06,07:03:58,LUCAS CUNHA DE OLIVEIRA,agua mineral,aguardando pagamento,Viçosa,MG,Brasil,36570-000,9.0,TRANSFERENCIA BANCARIA,MICAEL MALAQUIAS DE SOUZA OLIVEIRA,Minalba,10,1.40000,23.000000,14.000000


In [222]:
display(df[['status']].head(20))


,status
0,aguardando pagamento
1,aguardando pagamento
2,aguardando pagamento
3,aguardando pgto
4,pagamento confirmado
5,pagamento confirmado
6,pagamento confirmado
7,pagamento confirmado
8,aguardando pagamento
9,aguardando pagamento


In [223]:
pagamentos ={
    'aguardando pgto': 'aguardando pagamento',
    'boleto' : 'BOLETO',
    'pix' : 'PIX',
    'CARTAO DE CREDITO' :  'CARTAO CREDITO',
    'BOLETO NO DINHEIRO' : 'BOLETO'
}
df["pagamento"] = df["pagamento"].replace(pagamentos)
df["pagamento"] = df["pagamento"].str.strip()
df["pagamento"].unique()

array(['CARTAO CREDITO', 'TRANSFERENCIA BANCARIA', 'PIX', 'DOC', 'BOLETO',
       'BOLETO A VISTA', 'CARTAO CREDITO A VISTA', 'PIX A VISTA', 'TED',
       'PAGAMENTO INSTANTANEO'], dtype=object)

ajustes na forma de pagamento que estava com dados escritos errados 

In [224]:
df['produto'] = df['produto'].str.strip()  

In [225]:
df.to_csv('dados_corrigidos.csv', index=False)

In [226]:
display(df[['produto']].head(10))

,produto
0,macarrao
1,pasta de dente
2,sabonete
3,sabonete
4,sabonete
5,detergente
6,macarrao
7,refrigerante
8,acucar
9,agua mineral


In [227]:
df['produto'] = df['produto'].str.upper() 
df['produto'] = df['produto'].str.replace('-', ' ')  
df['produto'] = df['produto'].str.replace(r'\s+', ' ', regex=True) 

tirando caracteres especiais, espaços e transformando tudo para maiusculo

In [228]:
display(df[['produto']].head(30))

,produto
0,MACARRAO
1,PASTA DE DENTE
2,SABONETE
3,SABONETE
4,SABONETE
5,DETERGENTE
6,MACARRAO
7,REFRIGERANTE
8,ACUCAR
9,AGUA MINERAL


In [229]:
df.to_csv('dados_corrigidos.csv', index=False)

REGRAS DE ASSOCIACAO

NESSA PARTE ESTOU FAZENDO TODAS AS REGRAS DE ASSOCIACOES PARA DESCOBRIR QUAIS ITENS VENDE JUNTO A OUTROS

In [230]:
dados = pd.read_csv("dados_corrigidos.csv")

In [231]:
cestas = dados.groupby("id_da_compra")["produto"].apply(set).tolist()

In [232]:
codificador = TransactionEncoder()
cestas_codificadas = codificador.fit_transform(cestas)
base_binaria = pd.DataFrame(cestas_codificadas, columns=codificador.columns_)

In [233]:
conjuntos_frequentes = fpgrowth(base_binaria, min_support=0.02, use_colnames=True)

In [234]:
regras = association_rules(conjuntos_frequentes, metric="lift", min_threshold=1.0)

In [235]:
regras["id"] = range(1, len(regras) + 1)
regras_formatadas = regras[["id", "antecedents", "consequents", "support", "confidence", "lift"]]

In [236]:
regras_fortes = regras_formatadas[
    (regras_formatadas["lift"] > 2.0) & 
    (regras_formatadas["confidence"] > 0.7)
].sort_values(by=["lift", "confidence"], ascending=False)

In [237]:
print("Top 15 Regras Fortes de Associação:\n")
print(regras_fortes.head(15).to_string(index=False))

Top 15 Regras Fortes de Associação:

 id                      antecedents        consequents  support  confidence     lift
408                         (CARVAO)          (CERVEJA) 0.024050    1.000000 8.915156
264                  (ARROZ, FEIJAO)  (MOLHO DE TOMATE) 0.022800    0.825090 4.874419
259                          (ARROZ)  (MOLHO DE TOMATE) 0.062568    0.733633 4.334112
209         (LEITE INTEGRAL, ACUCAR)             (CAFE) 0.032384    0.930556 3.672943
363  (PAPEL TOALHA, PAPEL HIGIENICO)     (DESINFETANTE) 0.042367    0.795867 3.654618
365  (PAPEL HIGIENICO, DESINFETANTE)     (PAPEL TOALHA) 0.042367    0.800882 3.654432
253                   (CAFE, FEIJAO)           (ACUCAR) 0.036334    0.898969 3.491536
355      (AMACIANTE, PASTA DE DENTE)         (SABONETE) 0.038084    0.939556 3.357699
173         (MANTEIGA, PAO DE FORMA) (QUEIJO MUSSARELA) 0.050434    0.882988 2.982850
340             (MANTEIGA, PRESUNTO) (QUEIJO MUSSARELA) 0.048801    0.874552 2.954352
131 (QUEIJO MUSSA

In [238]:
def recomendar_produtos(produto_base, regras_df):
    print(f"\nSugestões baseadas em '{produto_base}':\n")
    
    matches = regras_df[regras_df["antecedents"].apply(lambda x: produto_base in x)]
    matches = matches.sort_values(by="lift", ascending=False)

    if matches.empty:
        print("Nenhuma sugestão encontrada.")
    else:
        for _, linha in matches.iterrows():
            recomendados = ", ".join(linha["consequents"])
            print(f"- {recomendados} | confiança: {linha['confidence']:.2f} | lift: {linha['lift']:.2f}")


Essa função pega um produto base e busca nas regras de associação outros produtos que costumam ser comprados junto com ele. Aí ela mostra essas sugestões com os valores de confiança e lift, que indicam a força da associação.

In [239]:

produtos_mais_vendidos = df.groupby('produto')['quantidade'].sum().sort_values(ascending=False).head(10)


print("10 produtos mais vendidos:")
print(produtos_mais_vendidos)


10 produtos mais vendidos:
produto
PASTA DE DENTE      210440
QUEIJO MUSSARELA    209052
SABONETE            198734
MANTEIGA            194397
PAPEL TOALHA        150377
DESINFETANTE        147913
CERVEJA             123498
MOLHO DE TOMATE     100124
CONDICIONADOR        92130
MACARRAO             76783
Name: quantidade, dtype: int64


exibindo quantas vezes o item foi vendido ,tanto sozinho mas em conjunto a outros

In [240]:
recomendar_produtos("PASTA DE DENTE", regras_fortes)



Sugestões baseadas em 'PASTA DE DENTE':

- SABONETE | confiança: 0.94 | lift: 3.36
